In [1]:
import pandas as pd
import numpy as np
from core.checker import parameters_checker
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from core.utils.utils import measurement_inverter, scaler, calculate_likelihood, calculate_entropy, calculate_multistep_loss
from numpy.linalg import eigvals
import nlopt


In [17]:
from core.estimator import estimator, selector, preparator
from core.creator import creator, initialiser, architector, filler
from core.utils.ic import ic_function

from smooth.adam_general._adam_general import adam_fitter, adam_forecaster

import warnings

In [3]:
%load_ext rpy2.ipython

/home/filtheo/smooth/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/filtheo/smooth/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [4]:
# Generate random monthly time series data
np.random.seed(42)  # For reproducibility
n_points = 24  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)


In [21]:
%%R -i ts_df
library(smooth)

model <- adam(ts_df, model = "ANN", lags = c(12))

forecast(model, h = 12)
model$B

   alpha    level 
 1.00000 51.99987 

/home/filtheo/smooth/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:56: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [9]:
%%R -i ts_df
library(smooth)
# Initialize variables matching R adam() function defaults
data <- ts_df
lags <- c(12)
model <- "ANN"
orders <- list(ar=c(0), i=c(0), ma=c(0), select=FALSE)
constant <- FALSE
formula <- NULL 
regressors <- 'use'
occurrence <- "none"
distribution <- "default"
loss <- "likelihood"
outliers <- "ignore"
level <- 0.99
h <- 0
holdout <- FALSE
persistence <- NULL
phi <- NULL
initial <- "optimal"
arma <- NULL
ic <- "AICc"
bounds <- "usual"
silent <- TRUE
# paste0() is needed in order to get rid of potential issues with names
yName <- paste0(deparse(substitute(data)),collapse="");

modelDo <- ""
ellipsis <- list();
# Assume that the model is not provided
profilesRecentProvided <- FALSE;
profilesRecentTable <- NULL;

# paste0() is needed in order to get rid of potential issues with names
yName <- paste0(deparse(substitute(data)),collapse="");
checkerReturn <- smooth:::parametersChecker(data, model, lags, formula, orders, constant, arma,
                                       outliers, level,
                                       persistence, phi, initial,
                                       distribution, loss, h, holdout, occurrence, ic, bounds,
                                       regressors, yName,
                                       silent, modelDo, ParentEnvironment=environment(), ellipsis, fast=FALSE);

cat("ets_model:", etsModel, "\n")
cat("e_type:", Etype, "\n")
cat("t_type:", Ttype, "\n")
cat("s_type:", Stype, "\n")
cat("model_is_trendy:", modelIsTrendy, "\n")
cat("model_is_seasonal:", modelIsSeasonal, "\n")
cat("lags:", paste(lags, collapse=", "), "\n")
cat("lags_model:", paste(lagsModel, collapse=", "), "\n")
cat("lags_model_arima:", paste(lagsModelARIMA, collapse=", "), "\n")
cat("lags_model_all:", paste(lagsModelAll, collapse=", "), "\n")
cat("lags_model_max:", lagsModelMax, "\n")
cat("profiles_recent_table:", paste(profilesRecentTable, collapse=", "), "\n")
cat("profiles_recent_provided:", profilesRecentProvided, "\n")
cat("obs_states:", obsStates, "\n")
cat("obs_in_sample:", obsInSample, "\n")
cat("obs_all:", obsAll, "\n")
cat("components_number_ets:", componentsNumberETS, "\n")
cat("components_number_ets_seasonal:", componentsNumberETSSeasonal, "\n")
cat("components_names_ets:", paste(componentsNamesETS, collapse=", "), "\n")
cat("ot_logical:", paste(otLogical, collapse=", "), "\n")
cat("y_in_sample:", paste(yInSample, collapse=", "), "\n")
cat("persistence:", paste(persistence, collapse=", "), "\n")
cat("persistence_estimate:", persistenceEstimate, "\n")
cat("persistence_level:", persistenceLevel, "\n")
cat("persistence_level_estimate:", persistenceLevelEstimate, "\n")
cat("persistence_trend:", persistenceTrend, "\n")
cat("persistence_trend_estimate:", persistenceTrendEstimate, "\n")
cat("persistence_seasonal:", persistenceSeasonal, "\n")
cat("persistence_seasonal_estimate:", persistenceSeasonalEstimate, "\n")
cat("persistence_xreg:", paste(persistenceXreg, collapse=", "), "\n")
cat("persistence_xreg_estimate:", persistenceXregEstimate, "\n")
cat("persistence_xreg_provided:", persistenceXregProvided, "\n")
cat("phi:", phi, "\n")
cat("initial_type:", initialType, "\n")
cat("initial_estimate:", initialEstimate, "\n")
cat("initial_level:", initialLevel, "\n")
cat("initial_level_estimate:", initialLevelEstimate, "\n")
cat("initial_trend:", initialTrend, "\n")
cat("initial_trend_estimate:", initialTrendEstimate, "\n")
cat("initial_seasonal:", paste(initialSeasonal, collapse=", "), "\n")
cat("initial_seasonal_estimate:", initialSeasonalEstimate, "\n")
cat("initial_arima:", paste(initialArima, collapse=", "), "\n")
cat("initial_arima_estimate:", initialArimaEstimate, "\n")
cat("initial_arima_number:", initialArimaNumber, "\n")
cat("initial_xreg_estimate:", initialXregEstimate, "\n")
cat("initial_xreg_provided:", initialXregProvided, "\n")
cat("arima_model:", arimaModel, "\n")
cat("ar_required:", arRequired, "\n")
cat("i_required:", iRequired, "\n")
cat("ma_required:", maRequired, "\n")
cat("arma_parameters:", paste(armaParameters, collapse=", "), "\n")
cat("ar_orders:", paste(arOrders, collapse=", "), "\n")
cat("i_orders:", paste(iOrders, collapse=", "), "\n")
cat("ma_orders:", paste(maOrders, collapse=", "), "\n")
cat("components_number_arima:", componentsNumberARIMA, "\n")
cat("components_names_arima:", paste(componentsNamesARIMA, collapse=", "), "\n")
cat("xreg_model:", xregModel, "\n")
cat("xreg_model_initials:", paste(xregModelInitials, collapse=", "), "\n")
cat("xreg_data:", paste(xregData, collapse=", "), "\n")
cat("xreg_number:", xregNumber, "\n")
cat("xreg_names:", paste(xregNames, collapse=", "), "\n")
cat("xreg_parameters_persistence:", paste(xregParametersPersistence, collapse=", "), "\n")
cat("constant_required:", constantRequired, "\n")
cat("constant_estimate:", constantEstimate, "\n")
cat("constant_value:", constantValue, "\n")
cat("constant_name:", constantName, "\n")


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:56: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():
R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v1.0.5 loaded.


R[write to console]: This is package "smooth", v3.1.6




ets_model: TRUE 
e_type: A 
t_type: N 
s_type: N 
model_is_trendy: FALSE 
model_is_seasonal: FALSE 
lags: 1 
lags_model: 1 
lags_model_arima: 0 
lags_model_all: 1 
lags_model_max: 1 
profiles_recent_table:  
profiles_recent_provided: FALSE 
obs_states: 25 
obs_in_sample: 24 
obs_all: 24 
components_number_ets: 1 
components_number_ets_seasonal: 0 
components_names_ets: level 
ot_logical: TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE 
y_in_sample: 52, 145, 160, 232, 293, 314, 397, 484, 559, 634, 722, 746, 749, 771, 824, 826, 914, 944, 982, 984, 1048, 1108, 1129, 1162 
persistence:  
persistence_estimate: TRUE 
persistence_level: 
persistence_level_estimate: TRUE 
persistence_trend: 
persistence_trend_estimate: FALSE 
persistence_seasonal: 
persistence_seasonal_estimate:  
persistence_xreg:  
persistence_xreg_estimate: FALSE 
persistence_xreg_provided: FALSE 
phi: 1 
initial_type: optimal 
in

In [10]:
%%R

if(modelDo=="estimate"){
    # If this is LASSO/RIDGE with lambda=1, use MSE to estimate initials only
    lambdaOriginal <- lambda;
    if(any(loss==c("LASSO","RIDGE")) && lambda==1){
        if(etsModel){
            # Pre-set ETS parameters
            persistenceEstimate[] <- FALSE;
            persistenceLevelEstimate[] <- persistenceTrendEstimate[] <-
                persistenceSeasonalEstimate[] <- FALSE;
            persistenceLevel <- persistenceTrend <- persistenceSeasonal <- 0;
            # Phi
            phiEstimate[] <- FALSE;
            phi <- 1;
        }
        if(xregModel){
            # ETSX parameters
            persistenceXregEstimate[] <- FALSE;
            persistenceXreg <- 0;
        }
        if(arimaModel){
            # Pre-set ARMA parameters
            arEstimate[] <- FALSE;
            maEstimate[] <- FALSE;
            armaParameters <- vector("numeric",sum(arOrders)+sum(maOrders));
            j <- 0;
            for(i in 1:length(lags)){
                if(arOrders[i]>0){
                    armaParameters[j+1:arOrders[i]] <- 1;
                    names(armaParameters)[j+c(1:arOrders[i])] <- paste0("phi",1:arOrders[i],"[",lags[i],"]");
                    j <- j + arOrders[i];
                }
                if(maOrders[i]>0){
                    armaParameters[j+1:maOrders[i]] <- 0;
                    names(armaParameters)[j+c(1:maOrders[i])] <- paste0("theta",1:maOrders[i],"[",lags[i],"]");
                    j <- j + maOrders[i];
                }
            }
        }
        lambda <- 0;
    }



}



In [11]:
%%R 

architector <- function(etsModel, Etype, Ttype, Stype, lags, lagsModelSeasonal,
                            xregNumber, obsInSample, initialType,
                            arimaModel, lagsModelARIMA, xregModel, constantRequired,
                            profilesRecentTable=NULL, profilesRecentProvided=FALSE){
        # If there is ETS
        if(etsModel){
            modelIsTrendy <- Ttype!="N";
            if(modelIsTrendy){
                # Make lags (1, 1)
                lagsModel <- matrix(c(1,1),ncol=1);
                componentsNamesETS <- c("level","trend");
            }
            else{
                # Make lags (1, ...)
                lagsModel <- matrix(c(1),ncol=1);
                componentsNamesETS <- c("level");
            }
            modelIsSeasonal <- Stype!="N";
            if(modelIsSeasonal){
                # If the lags are for the non-seasonal model
                lagsModel <- matrix(c(lagsModel,lagsModelSeasonal),ncol=1);
                componentsNumberETSSeasonal <- length(lagsModelSeasonal);
                if(componentsNumberETSSeasonal>1){
                    componentsNamesETS <- c(componentsNamesETS,paste0("seasonal",c(1:componentsNumberETSSeasonal)));
                }
                else{
                    componentsNamesETS <- c(componentsNamesETS,"seasonal");
                }
            }
            else{
                componentsNumberETSSeasonal <- 0;
            }
            lagsModelAll <- lagsModel;

            componentsNumberETS <- length(lagsModel);
        }
        else{
            modelIsTrendy <- modelIsSeasonal <- FALSE;
            componentsNumberETS <- componentsNumberETSSeasonal <- 0;
            componentsNamesETS <- NULL;
            lagsModelAll <- lagsModel <- NULL;
        }

        # If there is ARIMA
        if(arimaModel){
            lagsModelAll <- matrix(c(lagsModel,lagsModelARIMA), ncol=1);
        }

        # If constant is needed, add it
        if(constantRequired){
            lagsModelAll <- matrix(c(lagsModelAll,1), ncol=1);
        }

        # If there are xreg
        if(xregModel){
            lagsModelAll <- matrix(c(lagsModelAll,rep(1,xregNumber)), ncol=1);
        }

        lagsModelMax <- max(lagsModelAll);

        # Define the number of cols that should be in the matvt
        obsStates <- obsInSample + lagsModelMax;

        # Create ADAM profiles for correct treatment of seasonality
        adamProfiles <- adamProfileCreator(lagsModelAll, lagsModelMax, obsAll,
                                           lags=lags, yIndex=yIndexAll, yClasses=yClasses);
        if(profilesRecentProvided){
            profilesRecentTable <- profilesRecentTable[,1:lagsModelMax,drop=FALSE];
        }
        else{
            profilesRecentTable <- adamProfiles$recent;
        }
        indexLookupTable <- adamProfiles$lookup;

        return(list(lagsModel=lagsModel,lagsModelAll=lagsModelAll, lagsModelMax=lagsModelMax,
                    componentsNumberETS=componentsNumberETS, componentsNumberETSSeasonal=componentsNumberETSSeasonal,
                    componentsNumberETSNonSeasonal=componentsNumberETS-componentsNumberETSSeasonal,
                    componentsNamesETS=componentsNamesETS, obsStates=obsStates, modelIsTrendy=modelIsTrendy,
                    modelIsSeasonal=modelIsSeasonal,
                    indexLookupTable=indexLookupTable, profilesRecentTable=profilesRecentTable));
    }
    
    adamProfileCreator <- function(lagsModelAll, lagsModelMax, obsAll,
                               lags=NULL, yIndex=NULL, yClasses=NULL){
    # lagsModelAll - all lags used in the model for ETS + ARIMA + xreg
    # lagsModelMax - the maximum lag used in the model
    # obsAll - number of observations to create
    # lags - the original lags provided by user (no lags for ARIMA etc). Needed in order to see
    #        if weird frequencies are used.
    # yIndex - the indices needed in order to get the weird dates.
    # yClass - the class used for the actuals. If zoo, magic will happen here.
    # Create the matrix with profiles, based on provided lags
    profilesRecentTable <- matrix(0,length(lagsModelAll),lagsModelMax,
                                  dimnames=list(lagsModelAll,NULL));
    # Create the lookup table
    indexLookupTable <- matrix(1,length(lagsModelAll),obsAll+lagsModelMax,
                                    dimnames=list(lagsModelAll,NULL));
    # Modify the lookup table in order to get proper indices in C++
    profileIndices <- matrix(c(1:(lagsModelMax*length(lagsModelAll))),length(lagsModelAll));

    for(i in 1:length(lagsModelAll)){
        profilesRecentTable[i,1:lagsModelAll[i]] <- 1:lagsModelAll[i];
        # -1 is needed to align this with C++ code
        indexLookupTable[i,lagsModelMax+c(1:obsAll)] <- rep(profileIndices[i,1:lagsModelAll[i]],
                                                                 ceiling(obsAll/lagsModelAll[i]))[1:obsAll] -1;
        # Fix the head of the data, before the sample starts
        indexLookupTable[i,1:lagsModelMax] <- tail(rep(unique(indexLookupTable[i,lagsModelMax+c(1:obsAll)]),lagsModelMax),
                                                        lagsModelMax);
    }

    # Do shifts for proper lags only:
    # Check lags variable for 24 / 24*7 / 24*365 / 48 / 48*7 / 48*365 / 365 / 52
    # If they are there, find the DST / Leap moments
    # Then amend respective lookup values of profile, shifting them around
    if(any(yClasses=="zoo") && !is.null(yIndex) && !is.numeric(yIndex)){
        # If this is weekly data, duplicate 52, when 53 is used
        if(any(lags==52) && any(strftime(yIndex,format="%W")=="53")){
            shiftRows <- lagsModelAll==52;
            # If the data does not start with 1, proceed
            if(all(which(strftime(yIndex,format="%W")=="53")!=1)){
                indexLookupTable[shiftRows,which(strftime(yIndex,format="%W")=="53")] <-
                    indexLookupTable[shiftRows,which(strftime(yIndex,format="%W")=="53")-1];
            }
        }

        #### If this is daily and we have 365 days of year, locate 29th February and use 28th instead
        if(any(c(365,365*48,365*24) %in% lags) && any(strftime(yIndex,format="%d/%m")=="29/02")){
            shiftValue <- c(365,365*48,365*24)[c(365,365*48,365*24) %in% lags]/365;
            shiftRows <- lagsModelAll %in% c(365,365*48,365*24);
            # If the data does not start with 1/24/48, proceed (otherwise we refer to negative numbers)
            if(!any(which(strftime(yIndex,format="%d/%m")=="29/02") %in% shiftValue)){
                indexLookupTable[shiftRows,which(strftime(yIndex,format="%d/%m")=="29/02")] <-
                    indexLookupTable[shiftRows,which(strftime(yIndex,format="%d/%m")=="29/02")-shiftValue];
            }
        }

        #### If this is hourly; Locate DST and do shifts for specific observations
        if(any(c(24,24*7,24*365,48,48*7,48*365) %in% lags)){
            shiftRows <- lagsModelAll %in% c(24,48,24*7,48*7,24*365,48*365);
            # If this is hourly data, then shift 1 hour. If it is halfhourly, shift 2 hours
            shiftValue <- 1;
            if(any(c(48,48*7,48*365) %in% lags)){
                shiftValue[] <- 2;
            }
            # Get the start and the end of DST
            dstValues <- detectdst(yIndex);
            # If there are DST issues, do something
            doShifts <- !is.null(dstValues) && ((nrow(dstValues$start)!=0) | (nrow(dstValues$end)!=0))
            if(doShifts){
                # If the start date is not positioned before the end, introduce the artificial one
                if(nrow(dstValues$start)==0 ||
                   (nrow(dstValues$end)>0 && dstValues$start$id[1]>dstValues$end$id[1])){
                    dstValues$start <- rbind(data.frame(id=1,date=yIndex[1]),dstValues$start);
                }
                # If the end date is not present or the length of the end is not the same as the start,
                # set the end of series as one
                if(nrow(dstValues$end)==0 ||
                   nrow(dstValues$end)<nrow(dstValues$start)){
                    dstValues$end <- rbind(dstValues$end,data.frame(id=obsAll,date=tail(yIndex,1)));
                }
                # Shift everything from start to end dates by 1 obs forward.
                for(i in 1:nrow(dstValues$start)){
                    # If the end date is natural, just shift
                    if(dstValues$end$id[i]+shiftValue<=obsAll){
                        indexLookupTable[shiftRows,dstValues$start$id[i]:dstValues$end$id[i]] <-
                            indexLookupTable[shiftRows,dstValues$start$id[i]:dstValues$end$id[i]+shiftValue];
                    }
                    # If it isn't, we need to come up with the values for the end of sample
                    else{
                        indexLookupTable[shiftRows,dstValues$start$id[i]:dstValues$end$id[i]] <-
                            indexLookupTable[shiftRows,dstValues$start$id[i]:dstValues$end$id[i]-lagsModelMax+shiftValue];
                    }
                }
            }
        }
    }

    return(list(recent=profilesRecentTable,lookup=indexLookupTable));
}

adamArchitect <- architector(etsModel, Etype, Ttype, Stype, lags, lagsModelSeasonal,
                                     xregNumber, obsInSample, initialType,
                                     arimaModel, lagsModelARIMA, xregModel, constantRequired,
                                     profilesRecentTable, profilesRecentProvided)

adamArchitect

$lagsModel
     [,1]
[1,]    1

$lagsModelAll
     [,1]
[1,]    1

$lagsModelMax
[1] 1

$componentsNumberETS
[1] 1

$componentsNumberETSSeasonal
[1] 0

$componentsNumberETSNonSeasonal
[1] 1

$componentsNamesETS
[1] "level"

$obsStates
[1] 25

$modelIsTrendy
[1] FALSE

$modelIsSeasonal
[1] FALSE

$indexLookupTable
  [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
1    0    0    0    0    0    0    0    0    0     0     0     0     0     0
  [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
1     0     0     0     0     0     0     0     0     0     0     0

$profilesRecentTable
  [,1]
1    1



In [12]:
%%R

creator <- function(etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                        lags, lagsModel, lagsModelARIMA, lagsModelAll, lagsModelMax,
                        profilesRecentTable=NULL, profilesRecentProvided=FALSE,
                        obsStates, obsInSample, obsAll, componentsNumberETS, componentsNumberETSSeasonal,
                        componentsNamesETS, otLogical, yInSample,
                        # Persistence and phi
                        persistence, persistenceEstimate,
                        persistenceLevel, persistenceLevelEstimate, persistenceTrend, persistenceTrendEstimate,
                        persistenceSeasonal, persistenceSeasonalEstimate,
                        persistenceXreg, persistenceXregEstimate, persistenceXregProvided,
                        phi,
                        # Initials
                        initialType, initialEstimate,
                        initialLevel, initialLevelEstimate, initialTrend, initialTrendEstimate,
                        initialSeasonal, initialSeasonalEstimate,
                        initialArima, initialArimaEstimate, initialArimaNumber,
                        initialXregEstimate, initialXregProvided,
                        # ARIMA elements
                        arimaModel, arRequired, iRequired, maRequired, armaParameters,
                        arOrders, iOrders, maOrders,
                        componentsNumberARIMA, componentsNamesARIMA,
                        # Explanatory variables
                        xregModel, xregModelInitials, xregData, xregNumber, xregNames,
                        xregParametersPersistence,
                        # Constant
                        constantRequired, constantEstimate, constantValue, constantName){

        # Matrix of states. Time in columns, components in rows
        matVt <- matrix(NA, componentsNumberETS+componentsNumberARIMA+xregNumber+constantRequired, obsStates,
                        dimnames=list(c(componentsNamesETS,componentsNamesARIMA,xregNames,constantName),NULL));

        # Measurement rowvector
        matWt <- matrix(1, obsAll, componentsNumberETS+componentsNumberARIMA+xregNumber+constantRequired,
                        dimnames=list(NULL,c(componentsNamesETS,componentsNamesARIMA,xregNames,constantName)));

        # If xreg are provided, then fill in the respective values in Wt vector
        if(xregModel){
            matWt[,componentsNumberETS+componentsNumberARIMA+1:xregNumber] <- xregData;
        }

        # Transition matrix
        matF <- diag(componentsNumberETS+componentsNumberARIMA+xregNumber+constantRequired);

        # Persistence vector
        vecG <- matrix(0, componentsNumberETS+componentsNumberARIMA+xregNumber+constantRequired, 1,
                       dimnames=list(c(componentsNamesETS,componentsNamesARIMA,xregNames,constantName),NULL));

        j <- 0;
        # ETS model, persistence
        if(etsModel){
            j <- j+1;
            rownames(vecG)[j] <- "alpha";
            if(!persistenceLevelEstimate){
                vecG[j,] <- persistenceLevel;
            }
            if(modelIsTrendy){
                j <- j+1;
                rownames(vecG)[j] <- "beta";
                if(!persistenceTrendEstimate){
                    vecG[j,] <- persistenceTrend;
                }
            }
            if(modelIsSeasonal){
                if(!all(persistenceSeasonalEstimate)){
                    vecG[j+which(!persistenceSeasonalEstimate),] <- persistenceSeasonal;
                }
                if(componentsNumberETSSeasonal>1){
                    rownames(vecG)[j+c(1:componentsNumberETSSeasonal)] <- paste0("gamma",c(1:componentsNumberETSSeasonal));
                }
                else{
                    rownames(vecG)[j+1] <- "gamma";
                }
                j <- j+componentsNumberETSSeasonal;
            }
        }

        # ARIMA model, names for persistence
        if(arimaModel){
            # Remove diagonal from the ARIMA part of the matrix
            matF[j+1:componentsNumberARIMA,j+1:componentsNumberARIMA] <- 0;
            if(componentsNumberARIMA>1){
                rownames(vecG)[j+1:componentsNumberARIMA] <- paste0("psi",c(1:componentsNumberARIMA));
            }
            else{
                rownames(vecG)[j+1:componentsNumberARIMA] <- "psi";
            }
            j <- j+componentsNumberARIMA;
        }

        # Modify transition to do drift
        if(!arimaModel && constantRequired){
            matF[1,ncol(matF)] <- 1;
        }

        # Regression, persistence
        if(xregModel){
            if(persistenceXregProvided && !persistenceXregEstimate){
                vecG[j+1:xregNumber,] <- persistenceXreg;
            }
            rownames(vecG)[j+1:xregNumber] <- paste0("delta",xregParametersPersistence);
        }

        # Damping parameter value
        if(etsModel && modelIsTrendy){
            matF[1,2] <- phi;
            matF[2,2] <- phi;

            matWt[,2] <- phi;
        }

        # If the arma parameters were provided, fill in the persistence
        if(arimaModel && (!arEstimate && !maEstimate)){
            # Call polynomial
            # arimaPolynomials <- polynomialiser(NULL, arOrders, iOrders, maOrders,
            #                                    arRequired, maRequired, arEstimate, maEstimate, armaParameters, lags);
            arimaPolynomials <- lapply(adamPolynomialiser(0, arOrders, iOrders, maOrders,
                                                          arEstimate, maEstimate, armaParameters, lags), as.vector);
            # Fill in the transition matrix
            if(nrow(nonZeroARI)>0){
                matF[componentsNumberETS+nonZeroARI[,2],componentsNumberETS+nonZeroARI[,2]] <-
                    -arimaPolynomials$ariPolynomial[nonZeroARI[,1]];
            }
            # Fill in the persistence vector
            if(nrow(nonZeroARI)>0){
                vecG[componentsNumberETS+nonZeroARI[,2]] <- -arimaPolynomials$ariPolynomial[nonZeroARI[,1]];
            }
            if(nrow(nonZeroMA)>0){
                vecG[componentsNumberETS+nonZeroMA[,2]] <- vecG[componentsNumberETS+nonZeroMA[,2]] +
                    arimaPolynomials$maPolynomial[nonZeroMA[,1]];
            }
        }
        else{
            arimaPolynomials <- NULL;
        }


        if(!profilesRecentProvided){
            
            # ETS model, initial state
            # If something needs to be estimated...
            if(etsModel){
                if(initialEstimate){
                    # For the seasonal models
                    if(modelIsSeasonal){
                        if(obsNonzero>=lagsModelMax*2){
                            # If either Etype or Stype are multiplicative, do multiplicative decomposition
                            decompositionType <- c("additive","multiplicative")[any(c(Etype,Stype)=="M")+1];
                            yDecomposition <- msdecompose(yInSample, lags[lags!=1], type=decompositionType,
                                                          smoother=smoother);
                            j <- 1;
                            # level
                            if(initialLevelEstimate){
                                matVt[j,1:lagsModelMax] <- yDecomposition$initial[1];
                                # matVt[j,1:lagsModelMax] <-
                                #     switch(decompositionType,
                                #            "additive"=yDecomposition$initial[1]-yDecomposition$initial[2]*lagsModelMax,
                                #            "multiplicative"=yDecomposition$initial[1]/yDecomposition$initial[2]^lagsModelMax);
                                # matVt[j,1:lagsModelMax] <- mean(yInSample[1:lagsModelMax]);
                                if(xregModel){
                                    if(Etype=="A"){
                                        matVt[j,1:lagsModelMax] <- matVt[j,1:lagsModelMax] -
                                            as.vector(xregModelInitials[[1]]$initialXreg %*% xregData[1,]);
                                    }
                                    else{
                                        matVt[j,1:lagsModelMax] <- matVt[j,1:lagsModelMax] /
                                            as.vector(exp(xregModelInitials[[2]]$initialXreg %*% xregData[1,]));
                                    }
                                }
                            }
                            else{
                                matVt[j,1:lagsModelMax] <- initialLevel;
                            }
                            j <- j+1;
                            # If trend is needed
                            if(modelIsTrendy){
                                if(initialTrendEstimate){
                                    if(Ttype=="A" && Stype=="M"){
                                        # if(initialLevelEstimate){
                                        #     # level fix
                                        #     matVt[j-1,1:lagsModelMax] <- exp(mean(log(yInSample[otLogical][1:lagsModelMax])));
                                        # }
                                        # trend
                                        matVt[j,1:lagsModelMax] <- prod(yDecomposition$initial)-yDecomposition$initial[1];
                                        # If the initial trend is higher than the lowest value, initialise with zero.
                                        # This is a failsafe mechanism for the mixed models
                                        if(matVt[j,1]<0 && abs(matVt[j,1])>min(abs(yInSample[otLogical]))){
                                            matVt[j,1:lagsModelMax] <- 0;
                                        }
                                    }
                                    else if(Ttype=="M" && Stype=="A"){
                                        # if(initialLevelEstimate){
                                        #     # level fix
                                        #     matVt[j-1,1:lagsModelMax] <- exp(mean(log(yInSample[otLogical][1:lagsModelMax])));
                                        # }
                                        # trend
                                        matVt[j,1:lagsModelMax] <- sum(abs(yDecomposition$initial))/abs(yDecomposition$initial[1]);
                                    }
                                    else if(Ttype=="M"){
                                        # trend is too dangerous, make it start from 1.
                                        matVt[j,1:lagsModelMax] <- 1;
                                    }
                                    else{
                                        # trend
                                        matVt[j,1:lagsModelMax] <- yDecomposition$initial[2];
                                    }
                                    # This is a failsafe for multiplicative trend models, so that the thing does not explode
                                    if(Ttype=="M" && any(matVt[j,1:lagsModelMax]>1.1)){
                                        matVt[j,1:lagsModelMax] <- 1;
                                    }
                                    # This is a failsafe for multiplicative trend models, so that the thing does not explode
                                    if(Ttype=="M" && any(matVt[1,1:lagsModelMax]<0)){
                                        matVt[1,1:lagsModelMax] <- yInSample[otLogical][1];
                                    }
                                }
                                else{
                                    matVt[j,1:lagsModelMax] <- initialTrend;
                                }
                                j <- j+1;
                            }
                            #### Seasonal components
                            # For pure models use stuff as is
                            if(all(c(Etype,Stype)=="A") || all(c(Etype,Stype)=="M") ||
                               (Etype=="A" & Stype=="M")){
                                for(i in 1:componentsNumberETSSeasonal){
                                    if(initialSeasonalEstimate[i]){
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- yDecomposition$seasonal[[i]][1:lagsModel[i+j-1]];
                                        # Renormalise the initial seasons
                                        if(Stype=="A"){
                                            matVt[i+j-1,1:lagsModel[i+j-1]] <-
                                                matVt[i+j-1,1:lagsModel[i+j-1]] -
                                                mean(matVt[i+j-1,1:lagsModel[i+j-1]]);
                                        }
                                        else{
                                            matVt[i+j-1,1:lagsModel[i+j-1]] <-
                                                matVt[i+j-1,1:lagsModel[i+j-1]] /
                                                exp(mean(log(matVt[i+j-1,1:lagsModel[i+j-1]])));
                                        }
                                    }
                                    else{
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- initialSeasonal[[i]];
                                    }
                                }
                            }
                            # For mixed models use a different set of initials
                            else if(Etype=="M" && Stype=="A"){
                                for(i in 1:componentsNumberETSSeasonal){
                                    if(initialSeasonalEstimate[i]){
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <-
                                            log(yDecomposition$seasonal[[i]][1:lagsModel[i+j-1]])*min(yInSample[otLogical]);
                                        # Renormalise the initial seasons
                                        if(Stype=="A"){
                                            matVt[i+j-1,1:lagsModel[i+j-1]] <- matVt[i+j-1,1:lagsModel[i+j-1]] -
                                                mean(matVt[i+j-1,1:lagsModel[i+j-1]]);
                                        }
                                        else{
                                            matVt[i+j-1,1:lagsModel[i+j-1]] <- matVt[i+j-1,1:lagsModel[i+j-1]] /
                                                exp(mean(log(matVt[i+j-1,1:lagsModel[i+j-1]])));
                                        }
                                    }
                                    else{
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- initialSeasonal[[i]];
                                    }
                                }
                            }
                        }
                        else{
                            j <- 1;
                            # level
                            if(initialLevelEstimate){
                                matVt[j,1:lagsModelMax] <- mean(yInSample[1:lagsModelMax]);
                                if(xregModel){
                                    if(Etype=="A"){
                                        matVt[j,1:lagsModelMax] <- matVt[j,1:lagsModelMax] -
                                            as.vector(xregModelInitials[[1]]$initialXreg %*% xregData[1,]);
                                    }
                                    else{
                                        matVt[j,1:lagsModelMax] <- matVt[j,1:lagsModelMax] /
                                            as.vector(exp(xregModelInitials[[2]]$initialXreg %*% xregData[1,]));
                                    }
                                }
                            }
                            else{
                                matVt[j,1:lagsModelMax] <- initialLevel;
                            }
                            j <- j+1;
                            if(modelIsTrendy){
                                if(initialTrendEstimate){
                                    if(Ttype=="A"){
                                        # trend
                                        matVt[j,1:lagsModelMax] <- yInSample[2]-yInSample[1];
                                    }
                                    else if(Ttype=="M"){
                                        if(initialLevelEstimate){
                                            # level fix
                                            matVt[j-1,1:lagsModelMax] <- exp(mean(log(yInSample[otLogical][1:lagsModelMax])));
                                        }
                                        # trend
                                        matVt[j,1:lagsModelMax] <- yInSample[2]/yInSample[1];
                                    }
                                    # This is a failsafe for multiplicative trend models, so that the thing does not explode
                                    if(Ttype=="M" && any(matVt[j,1:lagsModelMax]>1.1)){
                                        matVt[j,1:lagsModelMax] <- 1;
                                    }
                                }
                                else{
                                    matVt[j,1:lagsModelMax] <- initialTrend;
                                }

                                # Do roll back. Especially useful for backcasting and multisteps
                                if(Ttype=="A"){
                                    matVt[j-1,1:lagsModelMax] <- matVt[j-1,1] - matVt[j,1]*lagsModelMax;
                                }
                                else if(Ttype=="M"){
                                    matVt[j-1,1:lagsModelMax] <- matVt[j-1,1] / matVt[j,1]^lagsModelMax;
                                }
                                j <- j+1;
                            }
                            #### Seasonal components
                            # For pure models use stuff as is
                            if(Stype=="A"){
                                for(i in 1:componentsNumberETSSeasonal){
                                    if(initialSeasonalEstimate[i]){
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- yInSample[1:lagsModel[i+j-1]]-matVt[1,1];
                                        # Renormalise the initial seasons
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- matVt[i+j-1,1:lagsModel[i+j-1]] -
                                            mean(matVt[i+j-1,1:lagsModel[i+j-1]]);
                                    }
                                    else{
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- initialSeasonal[[i]];
                                    }
                                }
                            }
                            # For mixed models use a different set of initials
                            else{
                                for(i in 1:componentsNumberETSSeasonal){
                                    if(initialSeasonalEstimate[i]){
                                        # abs() is needed for mixed ETS+ARIMA
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- yInSample[1:lagsModel[i+j-1]]/abs(matVt[1,1]);
                                        # Renormalise the initial seasons
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- matVt[i+j-1,1:lagsModel[i+j-1]] /
                                            exp(mean(log(matVt[i+j-1,1:lagsModel[i+j-1]])));
                                    }
                                    else{
                                        matVt[i+j-1,1:lagsModel[i+j-1]] <- initialSeasonal[[i]];
                                    }
                                }
                            }
                        }
                    }
                    # Non-seasonal models
                    else{
                        # level
                        if(initialLevelEstimate){
                            matVt[1,1:lagsModelMax] <- mean(yInSample[1:max(lagsModelMax,ceiling(obsInSample*0.2))]);
                            # if(xregModel){
                            #     if(Etype=="A"){
                            #         matVt[1,1:lagsModelMax] <- matVt[1,lagsModelMax] -
                            #             as.vector(xregModelInitials[[1]]$initialXreg %*% xregData[1,]);
                            #     }
                            #     else{
                            #         matVt[1,1:lagsModelMax] <- matVt[1,lagsModelMax] /
                            #             as.vector(exp(xregModelInitials[[2]]$initialXreg %*% xregData[1,]));
                            #     }
                            # }
                        }
                        else{
                            matVt[1,1:lagsModelMax] <- initialLevel;
                        }
                        if(modelIsTrendy){
                            if(initialTrendEstimate){
                                matVt[2,1:lagsModelMax] <- switch(Ttype,
                                                                  "A" = mean(diff(yInSample[1:max(lagsModelMax+1,
                                                                                                  ceiling(obsInSample*0.2))]),
                                                                             na.rm=TRUE),
                                                                  "M" = exp(mean(diff(log(yInSample[otLogical])),na.rm=TRUE)));
                            }
                            else{
                                matVt[2,1:lagsModelMax] <- initialTrend;
                            }
                        }
                    }

                    if(initialLevelEstimate && Etype=="M" && matVt[1,lagsModelMax]==0){
                        matVt[1,1:lagsModelMax] <- mean(yInSample);
                    }
                }
                # Else, insert the provided ones... make sure that this is not a backcasting
                else if(!initialEstimate && initialType=="provided"){
                    j <- 1;
                    matVt[j,1:lagsModelMax] <- initialLevel;
                    if(modelIsTrendy){
                        j <- j+1;
                        matVt[j,1:lagsModelMax] <- initialTrend;
                    }
                    if(modelIsSeasonal){
                        for(i in 1:componentsNumberETSSeasonal){
                            # This is misaligned, but that's okay, because this goes directly to profileRecent
                            # matVt[j+i,(lagsModelMax-lagsModel[j+i])+1:lagsModel[j+i]] <- initialSeasonal[[i]];
                            matVt[j+i,1:lagsModel[j+i]] <- initialSeasonal[[i]];
                        }
                    }
                    j <- j+componentsNumberETSSeasonal;
                }
            }

            # If ARIMA orders are specified, prepare initials
            if(arimaModel){
                if(initialArimaEstimate){
                    matVt[componentsNumberETS+1:componentsNumberARIMA, 1:initialArimaNumber] <-
                        switch(Etype, "A"=0, "M"=1);
                    if(any(lags>1)){
                        yDecomposition <- tail(msdecompose(yInSample,
                                                           lags[lags!=1],
                                                           type=switch(Etype,
                                                                       "A"="additive",
                                                                       "M"="multiplicative"))$seasonal,1)[[1]];
                    }
                    else{
                        yDecomposition <- switch(Etype,
                                                 "A"=mean(diff(yInSample[otLogical])),
                                                 "M"=exp(mean(diff(log(yInSample[otLogical])))));
                    }
                    matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber] <-
                        rep(yDecomposition,ceiling(initialArimaNumber/max(lags)))[1:initialArimaNumber];
                        # rep(yInSample[1:initialArimaNumber],each=componentsNumberARIMA);

                    # Failsafe mechanism in case the sample is too small
                    # matVt[is.na(matVt)] <- switch(Etype, "A"=0, "M"=1);

                    # If this is just ARIMA with optimisation, refine the initials
                    # if(!etsModel && initialType!="complete"){
                    #     arimaPolynomials <- polynomialiser(rep(0.1,sum(c(arOrders,maOrders))), arOrders, iOrders, maOrders,
                    #                                        arRequired, maRequired, arEstimate, maEstimate, armaParameters, lags);
                    #     if(nrow(nonZeroARI)>0 && nrow(nonZeroARI)>=nrow(nonZeroMA)){
                    #         matVt[componentsNumberETS+nonZeroARI[,2],
                    #               1:initialArimaNumber] <-
                    #             switch(Etype,
                    #                    "A"=arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #                        t(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]) /
                    #                        tail(arimaPolynomials$ariPolynomial,1),
                    #                    "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #                                t(log(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber])) /
                    #                                tail(arimaPolynomials$ariPolynomial,1)));
                    #     }
                    #     else{
                    #         matVt[componentsNumberETS+nonZeroMA[,2],
                    #               1:initialArimaNumber] <-
                    #             switch(Etype,
                    #                    "A"=arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                    #                        t(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]) /
                    #                        tail(arimaPolynomials$maPolynomial,1),
                    #                    "M"=exp(arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                    #                                t(log(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber])) /
                    #                                tail(arimaPolynomials$maPolynomial,1)));
                    #     }
                    # }
                }
                else{
                    # Fill in the matrix with 0 / 1, just in case if the state will not be updated anymore
                    matVt[componentsNumberETS+1:componentsNumberARIMA, 1:initialArimaNumber] <-
                        switch(Etype, "A"=0, "M"=1);
                    # Insert the provided initials
                    matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber] <-
                        initialArima[1:initialArimaNumber];

                    # matVt[componentsNumberETS+nonZeroARI[,2], 1:initialArimaNumber] <-
                    #     switch(Etype,
                    #            "A"=arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(initialArima[1:initialArimaNumber]) /
                    #                tail(arimaPolynomials$ariPolynomial,1),
                    #            "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(log(initialArima[1:initialArimaNumber])) /
                    #                        tail(arimaPolynomials$ariPolynomial,1)));

                    # If only AR is needed, but provided or if both are needed, but provided
                    # if(((arRequired && !arEstimate) && !maRequired) ||
                    #    ((arRequired && !arEstimate) && (maRequired && !maEstimate)) ||
                    #    (iRequired && !arEstimate && !maEstimate)){
                    #     matVt[componentsNumberETS+nonZeroARI[,2],1:initialArimaNumber] <-
                    #         switch(Etype,
                    #                "A"=arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #                    t(initialArima[1:initialArimaNumber]) /
                    #                    tail(arimaPolynomials$ariPolynomial,1),
                    #                "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #                            t(log(initialArima[1:initialArimaNumber])) /
                    #                            tail(arimaPolynomials$ariPolynomial,1)));
                    # }
                    # If only MA is needed, but provided
                    # else if(((maRequired && !maEstimate) && !arRequired)){
                    #     matVt[componentsNumberETS+nonZeroMA[,2],1:initialArimaNumber] <-
                    #         switch(Etype,
                    #                "A"=arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                    #                    t(initialArima[1:initialArimaNumber]) /
                    #                    tail(arimaPolynomials$maPolynomial,1),
                    #                "M"=exp(arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                    #                            t(log(initialArima[1:initialArimaNumber])) /
                    #                            tail(arimaPolynomials$maPolynomial,1)));
                    # }
                }
            }

            # Fill in the initials for xreg
            if(xregModel){
                if(Etype=="A" || initialXregProvided || is.null(xregModelInitials[[2]])){
                    matVt[componentsNumberETS+componentsNumberARIMA+1:xregNumber,
                          1:lagsModelMax] <- xregModelInitials[[1]]$initialXreg;
                }
                else{
                    matVt[componentsNumberETS+componentsNumberARIMA+1:xregNumber,
                          1:lagsModelMax] <- xregModelInitials[[2]]$initialXreg;
                }
            }

            # Add constant if needed
            if(constantRequired){
                if(constantEstimate){
                    # Add the mean of data
                    if(sum(iOrders)==0 && !etsModel){
                        matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,] <- mean(yInSample[otLogical]);
                    }
                    # Add first differences
                    else{
                        matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,] <-
                            switch(Etype,
                                   "A"=mean(diff(yInSample[otLogical])),
                                   "M"=exp(mean(diff(log(yInSample[otLogical])))));
                    }
                }
                else{
                    matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,] <- constantValue;
                }
                # If ETS model is used, change the initial level
                if(etsModel && initialLevelEstimate){
                    if(Etype=="A"){
                        matVt[1,1:lagsModelMax] <- matVt[1,1:lagsModelMax] -
                            matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,1];
                    }
                    else{
                        matVt[1,1:lagsModelMax] <- matVt[1,1:lagsModelMax] /
                            matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,1];
                    }
                }
                # If ARIMA is done, debias states
                if(arimaModel && initialArimaEstimate){
                    if(Etype=="A"){
                        matVt[componentsNumberETS+nonZeroARI[,2],1:initialArimaNumber] <-
                            matVt[componentsNumberETS+nonZeroARI[,2],1:initialArimaNumber] -
                            matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,1];
                    }
                    else{
                        matVt[componentsNumberETS+nonZeroARI[,2],1:initialArimaNumber] <-
                            matVt[componentsNumberETS+nonZeroARI[,2],1:initialArimaNumber] /
                            matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,1];
                    }
                }
            }
        }
        else{
            matVt[,1:lagsModelMax] <- profilesRecentTable;
        }

        return(list(matVt=matVt, matWt=matWt, matF=matF, vecG=vecG, arimaPolynomials=arimaPolynomials));
    }

adamCreated <- creator(etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                               lags, lagsModel, lagsModelARIMA, lagsModelAll, lagsModelMax,
                               profilesRecentTable, profilesRecentProvided,
                               obsStates, obsInSample, obsAll, componentsNumberETS, componentsNumberETSSeasonal,
                               componentsNamesETS, otLogical, yInSample,
                               persistence, persistenceEstimate,
                               persistenceLevel, persistenceLevelEstimate, persistenceTrend, persistenceTrendEstimate,
                               persistenceSeasonal, persistenceSeasonalEstimate,
                               persistenceXreg, persistenceXregEstimate, persistenceXregProvided,
                               phi,
                               initialType, initialEstimate,
                               initialLevel, initialLevelEstimate, initialTrend, initialTrendEstimate,
                               initialSeasonal, initialSeasonalEstimate,
                               initialArima, initialArimaEstimate, initialArimaNumber,
                               initialXregEstimate, initialXregProvided,
                               arimaModel, arRequired, iRequired, maRequired, armaParameters,
                               arOrders, iOrders, maOrders,
                               componentsNumberARIMA, componentsNamesARIMA,
                               xregModel, xregModelInitials, xregData, xregNumber, xregNames,
                               xregParametersPersistence,
                               constantRequired, constantEstimate, constantValue, constantName)

adamCreated$matVt


       [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
level 176.4   NA   NA   NA   NA   NA   NA   NA   NA    NA    NA    NA    NA
      [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
level    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA


In [13]:
%%R

initialiser <- function(etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                            componentsNumberETSNonSeasonal, componentsNumberETSSeasonal, componentsNumberETS,
                            lags, lagsModel, lagsModelSeasonal, lagsModelARIMA, lagsModelMax,
                            matVt,
                            # persistence values
                            persistenceEstimate, persistenceLevelEstimate, persistenceTrendEstimate,
                            persistenceSeasonalEstimate, persistenceXregEstimate,
                            # initials
                            phiEstimate, initialType, initialEstimate,
                            initialLevelEstimate, initialTrendEstimate, initialSeasonalEstimate,
                            initialArimaEstimate, initialXregEstimate,
                            # ARIMA elements
                            arimaModel, arRequired, maRequired, arEstimate, maEstimate, arOrders, maOrders,
                            componentsNumberARIMA, componentsNamesARIMA, initialArimaNumber,
                            # Explanatory variables
                            xregModel, xregNumber,
                            xregParametersEstimated, xregParametersPersistence,
                            # Constant and other stuff
                            constantEstimate, constantName, otherParameterEstimate){
        # The vector of logicals for persistence elements
        persistenceEstimateVector <- c(persistenceLevelEstimate,modelIsTrendy&persistenceTrendEstimate,
                                       modelIsSeasonal&persistenceSeasonalEstimate);

        # The order:
        # Persistence of states and for xreg, phi, AR and MA parameters, initials, initialsARIMA, initials for xreg
        B <- Bl <- Bu <- vector("numeric",
                                # Values of the persistence vector + phi
                                etsModel*(persistenceLevelEstimate + modelIsTrendy*persistenceTrendEstimate +
                                              modelIsSeasonal*sum(persistenceSeasonalEstimate) + phiEstimate) +
                                    xregModel*persistenceXregEstimate*max(xregParametersPersistence) +
                                    # AR and MA values
                                    arimaModel*(arEstimate*sum(arOrders)+maEstimate*sum(maOrders)) +
                                    # initials of ETS
                                    etsModel*all(initialType!=c("complete","backcasting"))*
                                    (initialLevelEstimate +
                                         (modelIsTrendy*initialTrendEstimate) +
                                         (modelIsSeasonal*sum(initialSeasonalEstimate*(lagsModelSeasonal-1)))) +
                                    # initials of ARIMA
                                    all(initialType!=c("complete","backcasting"))*arimaModel*initialArimaNumber*initialArimaEstimate +
                                    # initials of xreg
                                    (initialType!="complete")*xregModel*initialXregEstimate*sum(xregParametersEstimated) +
                                    constantEstimate + otherParameterEstimate);

        j <- 0;
        if(etsModel){
            # Fill in persistence
            if(persistenceEstimate && any(persistenceEstimateVector)){
                if(any(c(Etype,Ttype,Stype)=="M")){
                    # A special type of model which is not safe: AAM, MAA, MAM
                    if((Etype=="A" && Ttype=="A" && Stype=="M") || (Etype=="A" && Ttype=="M" && Stype=="A") ||
                       (any(initialType==c("complete","backcasting")) &&
                        ((Etype=="M" && Ttype=="A" && Stype=="A") || (Etype=="M" && Ttype=="A" && Stype=="M")))){
                        B[1:sum(persistenceEstimateVector)] <-
                            c(0.01,0,rep(0,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                    }
                    # MMA is the worst. Set everything to zero and see if anything can be done...
                    else if((Etype=="M" && Ttype=="M" && Stype=="A")){
                        B[1:sum(persistenceEstimateVector)] <-
                            c(0,0,rep(0,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                    }
                    else if(Etype=="M" && Ttype=="A"){
                        if(any(initialType==c("complete","backcasting"))){
                            B[1:sum(persistenceEstimateVector)] <-
                                c(0.1,0,rep(0.3,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                        }
                        else{
                            B[1:sum(persistenceEstimateVector)] <-
                                c(0.2,0.01,rep(0.3,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                        }
                    }
                    else if(Etype=="M" && Ttype=="M"){
                        B[1:sum(persistenceEstimateVector)] <-
                            c(0.1,0.05,rep(0.3,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                    }
                    else{
                        B[1:sum(persistenceEstimateVector)] <-
                            c(0.1,0.05,rep(0.3,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                    }
                }
                else{
                    B[1:sum(persistenceEstimateVector)] <-
                        c(0.1,0.05,rep(0.3,componentsNumberETSSeasonal))[which(persistenceEstimateVector)];
                }
                if(bounds=="usual"){
                    Bl[1:sum(persistenceEstimateVector)] <- rep(0, sum(persistenceEstimateVector));
                    Bu[1:sum(persistenceEstimateVector)] <- rep(1, sum(persistenceEstimateVector));
                }
                else{
                    Bl[1:sum(persistenceEstimateVector)] <- rep(-5, sum(persistenceEstimateVector));
                    Bu[1:sum(persistenceEstimateVector)] <- rep(5, sum(persistenceEstimateVector));
                }
                # Names for B
                if(persistenceLevelEstimate){
                    j[] <- j+1
                    names(B)[j] <- "alpha";
                }
                if(modelIsTrendy && persistenceTrendEstimate){
                    j[] <- j+1
                    names(B)[j] <- "beta";
                }
                if(modelIsSeasonal && any(persistenceSeasonalEstimate)){
                    if(componentsNumberETSSeasonal>1){
                        names(B)[j+c(1:sum(persistenceSeasonalEstimate))] <-
                            paste0("gamma",c(1:componentsNumberETSSeasonal));
                    }
                    else{
                        names(B)[j+1] <- "gamma";
                    }
                    j[] <- j+sum(persistenceSeasonalEstimate);
                }
            }
        }

        # Persistence if xreg is provided
        if(xregModel && persistenceXregEstimate){
            xregPersistenceNumber <- max(xregParametersPersistence);
            B[j+1:xregPersistenceNumber] <- rep(switch(Etype,"A"=0.01,"M"=0),xregPersistenceNumber);
            Bl[j+1:xregPersistenceNumber] <- rep(-5, xregPersistenceNumber);
            Bu[j+1:xregPersistenceNumber] <- rep(5, xregPersistenceNumber);
            names(B)[j+1:xregPersistenceNumber] <- paste0("delta",c(1:xregPersistenceNumber));
            j[] <- j+xregPersistenceNumber;
        }

        # Damping parameter
        if(etsModel && phiEstimate){
            j[] <- j+1;
            B[j] <- 0.95;
            names(B)[j] <- "phi";
            Bl[j] <- 0;
            Bu[j] <- 1;
        }

        # ARIMA parameters (AR / MA)
        if(arimaModel){
            # These are filled in lags-wise
            if(any(c(arEstimate,maEstimate))){
                acfValues <- rep(-0.1, maOrders %*% lags);
                pacfValues <- rep(0.1, arOrders %*% lags);
                # If this is ETS + ARIMA model or no differences model, then don't bother with initials
                # The latter does not make sense because of non-stationarity in ACF / PACF
                # Otherwise use ACF / PACF values as starting parameters for ARIMA
                if(!(etsModel || all(iOrders==0))){
                    yDifferenced <- yInSample;
                    # If the model has differences, take them
                    if(any(iOrders>0)){
                        for(i in 1:length(iOrders)){
                            if(iOrders[i]>0){
                                yDifferenced <- diff(yDifferenced,lag=lags[i],differences=iOrders[i]);
                            }
                        }
                    }
                    # Do ACF/PACF initialisation only for non-seasonal models
                    if(all(lags<=1)){
                        if(maRequired && maEstimate){
                            # If the sample is smaller than lags, it will be substituted by default values
                            acfValues[1:min(maOrders %*% lags, length(yDifferenced)-1)] <-
                                acf(yDifferenced,lag.max=max(1,maOrders %*% lags),plot=FALSE)$acf[-1];
                        }
                        if(arRequired && arEstimate){
                            # If the sample is smaller than lags, it will be substituted by default values
                            pacfValues[1:min(arOrders %*% lags, length(yDifferenced)-1)] <-
                                pacf(yDifferenced,lag.max=max(1,arOrders %*% lags),plot=FALSE)$acf;
                        }
                    }
                }
                for(i in 1:length(lags)){
                    if(arRequired && arEstimate && arOrders[i]>0){
                        if(all(!is.nan(pacfValues[c(1:arOrders[i])*lags[i]]))){
                            B[j+c(1:arOrders[i])] <- pacfValues[c(1:arOrders[i])*lags[i]];
                        }
                        else{
                            B[j+c(1:arOrders[i])] <- 0.1;
                        }
                        if(sum(B[j+c(1:arOrders[i])])>1){
                            B[j+c(1:arOrders[i])] <- B[j+c(1:arOrders[i])] / sum(B[j+c(1:arOrders[i])]) - 0.01;
                        }
                        # B[j+c(1:arOrders[i])] <- rep(0.1,arOrders[i]);
                        Bl[j+c(1:arOrders[i])] <- -5;
                        Bu[j+c(1:arOrders[i])] <- 5;
                        names(B)[j+1:arOrders[i]] <- paste0("phi",1:arOrders[i],"[",lags[i],"]");
                        j[] <- j + arOrders[i];
                    }
                    if(maRequired && maEstimate && maOrders[i]>0){
                        if(all(!is.nan(acfValues[c(1:maOrders[i])*lags[i]]))){
                            B[j+c(1:maOrders[i])] <- acfValues[c(1:maOrders[i])*lags[i]];
                        }
                        else{
                            B[j+c(1:maOrders[i])] <- 0.1;
                        }
                        if(sum(B[j+c(1:maOrders[i])])>1){
                            B[j+c(1:maOrders[i])] <- B[j+c(1:maOrders[i])] / sum(B[j+c(1:maOrders[i])]) - 0.01;
                        }
                        # B[j+c(1:maOrders[i])] <- rep(-0.1,maOrders[i]);
                        Bl[j+c(1:maOrders[i])] <- -5;
                        Bu[j+c(1:maOrders[i])] <- 5;
                        names(B)[j+1:maOrders[i]] <- paste0("theta",1:maOrders[i],"[",lags[i],"]");
                        j[] <- j + maOrders[i];
                    }
                }
            }

            arimaPolynomials <- lapply(adamPolynomialiser(B[j+1:sum(c(arOrders*arEstimate,maOrders*maEstimate))],
                                                          arOrders, iOrders, maOrders,
                                                          arEstimate, maEstimate, armaParameters, lags), as.vector)
        }

        # Initials
        if(etsModel && all(initialType!=c("complete","backcasting")) && initialEstimate){
            if(initialLevelEstimate){
                j[] <- j+1;
                B[j] <- matVt[1,1];
                names(B)[j] <- "level";
                if(Etype=="A"){
                    Bl[j] <- -Inf;
                    Bu[j] <- Inf;
                }
                else{
                    Bl[j] <- 0;
                    Bu[j] <- Inf;
                }
            }
            if(modelIsTrendy && initialTrendEstimate){
                j[] <- j+1;
                B[j] <- matVt[2,1];
                names(B)[j] <- "trend";
                if(Ttype=="A"){
                    Bl[j] <- -Inf;
                    Bu[j] <- Inf;
                }
                else{
                    Bl[j] <- 0;
                    # 2 is already too much for the multiplicative model
                    Bu[j] <- 2;
                }
            }
            if(modelIsSeasonal && any(initialSeasonalEstimate)){
                if(componentsNumberETSSeasonal>1){
                    for(k in 1:componentsNumberETSSeasonal){
                        if(initialSeasonalEstimate[k]){
                            # -1 is needed in order to remove the redundant seasonal element (normalisation)
                            B[j+2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <-
                                matVt[componentsNumberETSNonSeasonal+k, 2:lagsModel[componentsNumberETSNonSeasonal+k]-1];
                            names(B)[j+2:(lagsModel[componentsNumberETSNonSeasonal+k])-1] <-
                                paste0("seasonal",k,"_",2:lagsModel[componentsNumberETSNonSeasonal+k]-1);
                            if(Stype=="A"){
                                Bl[j+2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <- -Inf;
                                Bu[j+2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <- Inf;
                            }
                            else{
                                Bl[j+2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <- 0;
                                Bu[j+2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <- Inf;
                            }
                            j[] <- j+(lagsModelSeasonal[k]-1);
                        }
                    }
                }
                else{
                    # -1 is needed in order to remove the redundant seasonal element (normalisation)
                    B[j+2:(lagsModel[componentsNumberETS])-1] <- matVt[componentsNumberETS,2:lagsModel[componentsNumberETS]-1];
                    names(B)[j+2:(lagsModel[componentsNumberETS])-1] <- paste0("seasonal_",2:lagsModel[componentsNumberETS]-1);
                    if(Stype=="A"){
                        Bl[j+2:(lagsModel[componentsNumberETS])-1] <- -Inf;
                        Bu[j+2:(lagsModel[componentsNumberETS])-1] <- Inf;
                    }
                    else{
                        Bl[j+2:(lagsModel[componentsNumberETS])-1] <- 0;
                        Bu[j+2:(lagsModel[componentsNumberETS])-1] <- Inf;
                    }
                    j[] <- j+(lagsModel[componentsNumberETS]-1);
                }
            }
        }

        # ARIMA initials
        if(arimaModel && all(initialType!=c("complete","backcasting")) && initialArimaEstimate){
            B[j+1:initialArimaNumber] <- head(matVt[componentsNumberETS+componentsNumberARIMA,1:lagsModelMax],initialArimaNumber);
            names(B)[j+1:initialArimaNumber] <- paste0("ARIMAState",1:initialArimaNumber);

            # Fix initial state if the polynomial is not zero
            if(tail(arimaPolynomials$ariPolynomial,1)!=0){
                B[j+1:initialArimaNumber] <- B[j+1:initialArimaNumber] / tail(arimaPolynomials$ariPolynomial,1);
            }

            if(Etype=="A"){
                Bl[j+1:initialArimaNumber] <- -Inf;
                Bu[j+1:initialArimaNumber] <- Inf;
            }
            else{
                # Make sure that ARIMA states are positive to avoid errors
                B[j+1:initialArimaNumber] <- abs(B[j+1:initialArimaNumber]);
                Bl[j+1:initialArimaNumber] <- 0;
                Bu[j+1:initialArimaNumber] <- Inf;
            }
            j[] <- j+initialArimaNumber;
        }

        # Initials of the xreg
        if(initialType!="complete" && initialXregEstimate){
            xregNumberToEstimate <- sum(xregParametersEstimated);
            B[j+1:xregNumberToEstimate] <- matVt[componentsNumberETS+componentsNumberARIMA+
                                                     which(xregParametersEstimated==1),1];
            names(B)[j+1:xregNumberToEstimate] <- rownames(matVt)[componentsNumberETS+componentsNumberARIMA+
                                                                      which(xregParametersEstimated==1)];
            if(Etype=="A"){
                Bl[j+1:xregNumberToEstimate] <- -Inf;
                Bu[j+1:xregNumberToEstimate] <- Inf;
            }
            else{
                Bl[j+1:xregNumberToEstimate] <- -Inf;
                Bu[j+1:xregNumberToEstimate] <- Inf;
            }
            j[] <- j+xregNumberToEstimate;
        }

        if(constantEstimate){
            j[] <- j+1;
            B[j] <- matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,1];
            names(B)[j] <- constantName;
            if(etsModel || sum(iOrders)!=0){
                if(Etype=="A"){
                    Bu[j] <- quantile(diff(yInSample[otLogical]),0.6);
                    Bl[j] <- -Bu[j];
                }
                else{
                    Bu[j] <- exp(quantile(diff(log(yInSample[otLogical])),0.6));
                    Bl[j] <- exp(quantile(diff(log(yInSample[otLogical])),0.4));
                }

                # Failsafe for weird cases, when upper bound is the same or lower than the lower one
                if(Bu[j]<=Bl[j]){
                    Bu[j] <- Inf;
                    Bl[j] <- switch(Etype,"A"=-Inf,"M"=0);
                }

                # Failsafe for cases, when the B is outside of bounds
                if(B[j]<=Bl[j]){
                    Bl[j] <- switch(Etype,"A"=-Inf,"M"=0);
                }
                if(B[j]>=Bu[j]){
                    Bu[j] <- Inf;
                }
            }
            else{
                # if(Etype=="A"){
                    # B[j]*1.01 is needed to make sure that the bounds cover the initial value
                    Bu[j] <- max(abs(yInSample[otLogical]),abs(B[j])*1.01);
                    Bl[j] <- -Bu[j];
                # }
                # else{
                #     Bu[j] <- 1.5;
                #     Bl[j] <- 0.1;
                # }
                # If this is just a constant
            }
        }

        # Add lambda if it is needed
        if(otherParameterEstimate){
            j[] <- j+1;
            B[j] <- other;
            names(B)[j] <- "other";
            Bl[j] <- 1e-10;
            Bu[j] <- Inf;
        }

        return(list(B=B,Bl=Bl,Bu=Bu));
    }

BValues <- initialiser(etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                               componentsNumberETSNonSeasonal, componentsNumberETSSeasonal, componentsNumberETS,
                               lags, lagsModel, lagsModelSeasonal, lagsModelARIMA, lagsModelMax,
                               adamCreated$matVt,
                               persistenceEstimate, persistenceLevelEstimate, persistenceTrendEstimate,
                               persistenceSeasonalEstimate, persistenceXregEstimate,
                               phiEstimate, initialType, initialEstimate,
                               initialLevelEstimate, initialTrendEstimate, initialSeasonalEstimate,
                               initialArimaEstimate, initialXregEstimate,
                               arimaModel, arRequired, maRequired, arEstimate, maEstimate, arOrders, maOrders,
                               componentsNumberARIMA, componentsNamesARIMA, initialArimaNumber,
                               xregModel, xregNumber,
                               xregParametersEstimated, xregParametersPersistence,
                               constantEstimate, constantName, otherParameterEstimate)
BValues

$B
alpha level 
  0.1 176.4 

$Bl
[1]    0 -Inf

$Bu
[1]   1 Inf



In [14]:
%%R 

BValues

$B
alpha level 
  0.1 176.4 

$Bl
[1]    0 -Inf

$Bu
[1]   1 Inf



In [15]:
%%R 

matVt <- adamCreated$matVt
matWt <- adamCreated$matWt
matF <- adamCreated$matF
vecG <- adamCreated$vecG
arimaPolynomials <- adamCreated$arimaPolynomials
B <- BValues$B
filler <- function(B,
                       etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                       componentsNumberETS, componentsNumberETSNonSeasonal,
                       componentsNumberETSSeasonal, componentsNumberARIMA,
                       lags, lagsModel, lagsModelMax,
                       # The main matrices
                       matVt, matWt, matF, vecG,
                       # Persistence and phi
                       persistenceEstimate, persistenceLevelEstimate, persistenceTrendEstimate,
                       persistenceSeasonalEstimate, persistenceXregEstimate,
                       phiEstimate,
                       # Initials
                       initialType, initialEstimate,
                       initialLevelEstimate, initialTrendEstimate, initialSeasonalEstimate,
                       initialArimaEstimate, initialXregEstimate,
                       # ARIMA
                       arimaModel, arEstimate, maEstimate, arOrders, iOrders, maOrders,
                       arRequired, maRequired, armaParameters,
                       nonZeroARI, nonZeroMA, arimaPolynomials,
                       # Explanatory variables
                       xregModel, xregNumber,
                       xregParametersMissing, xregParametersIncluded,
                       xregParametersEstimated, xregParametersPersistence,
                       # Constant
                       constantEstimate){

        j <- 0;
        # Fill in persistence
        if(persistenceEstimate){
            # Persistence of ETS
            if(etsModel){
                i <- 1;
                # alpha
                if(persistenceLevelEstimate){
                    j[] <- j+1;
                    vecG[i] <- B[j];
                }
                # beta
                if(modelIsTrendy){
                    i[] <- 2;
                    if(persistenceTrendEstimate){
                        j[] <- j+1;
                        vecG[i] <- B[j];
                    }
                }
                # gamma1, gamma2, ...
                if(modelIsSeasonal){
                    if(any(persistenceSeasonalEstimate)){
                        vecG[i+which(persistenceSeasonalEstimate)] <- B[j+c(1:sum(persistenceSeasonalEstimate))];
                        j[] <- j+sum(persistenceSeasonalEstimate);
                    }
                    i[] <- componentsNumberETS;
                }
            }

            # Persistence of xreg
            if(xregModel && persistenceXregEstimate){
                xregPersistenceNumber <- max(xregParametersPersistence);
                vecG[j+componentsNumberARIMA+1:length(xregParametersPersistence)] <-
                    B[j+1:xregPersistenceNumber][xregParametersPersistence];
                j[] <- j+xregPersistenceNumber;
            }
        }

        # Damping parameter
        if(etsModel && phiEstimate){
            j[] <- j+1;
            matWt[,2] <- B[j];
            matF[1:2,2] <- B[j];
        }

        # ARMA parameters. This goes before xreg in persistence
        if(arimaModel){
            # Call the function returning ARI and MA polynomials
            # arimaPolynomials <- polynomialiser(B[j+1:sum(c(arOrders*arEstimate,maOrders*maEstimate))], arOrders, iOrders, maOrders,
            #                                    arRequired, maRequired, arEstimate, maEstimate, armaParameters, lags);
            arimaPolynomials <- lapply(adamPolynomialiser(B[j+1:sum(c(arOrders*arEstimate,maOrders*maEstimate))],
                                                          arOrders, iOrders, maOrders,
                                                          arEstimate, maEstimate, armaParameters, lags), as.vector);

            # Fill in the transition matrix
            if(nrow(nonZeroARI)>0){
                matF[componentsNumberETS+nonZeroARI[,2],componentsNumberETS+1:(componentsNumberARIMA+constantRequired)] <-
                    -arimaPolynomials$ariPolynomial[nonZeroARI[,1]];
            }
            # Fill in the persistence vector
            if(nrow(nonZeroARI)>0){
                vecG[componentsNumberETS+nonZeroARI[,2]] <- -arimaPolynomials$ariPolynomial[nonZeroARI[,1]];
            }
            if(nrow(nonZeroMA)>0){
                vecG[componentsNumberETS+nonZeroMA[,2]] <- vecG[componentsNumberETS+nonZeroMA[,2]] +
                    arimaPolynomials$maPolynomial[nonZeroMA[,1]];
            }
            j[] <- j+sum(c(arOrders*arEstimate,maOrders*maEstimate));
        }

        # Initials of ETS if something needs to be estimated
        if(etsModel && all(initialType!=c("complete","backcasting")) && initialEstimate){
            i <- 1;
            if(initialLevelEstimate){
                j[] <- j+1;
                matVt[i,1:lagsModelMax] <- B[j];
            }
            i[] <- i+1;
            if(modelIsTrendy && initialTrendEstimate){
                j[] <- j+1;
                matVt[i,1:lagsModelMax] <- B[j];
                i[] <- i+1;
            }
            if(modelIsSeasonal && any(initialSeasonalEstimate)){
                for(k in 1:componentsNumberETSSeasonal){
                    if(initialSeasonalEstimate[k]){
                        matVt[componentsNumberETSNonSeasonal+k, 2:lagsModel[componentsNumberETSNonSeasonal+k]-1] <-
                            B[j+2:(lagsModel[componentsNumberETSNonSeasonal+k])-1];
                        matVt[componentsNumberETSNonSeasonal+k, lagsModel[componentsNumberETSNonSeasonal+k]] <-
                            switch(Stype,
                                   "A"=-sum(B[j+2:(lagsModel[componentsNumberETSNonSeasonal+k])-1]),
                                   "M"=1/prod(B[j+2:(lagsModel[componentsNumberETSNonSeasonal+k])-1]));
                        j[] <- j+lagsModel[componentsNumberETSNonSeasonal+k]-1;
                    }
                }
            }
        }

        # Initials of ARIMA
        if(arimaModel){
            if(all(initialType!=c("complete","backcasting")) && initialArimaEstimate){
                # matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber] <- B[j+1:initialArimaNumber];
                # for(i in (componentsNumberARIMA-1):1){
                #     indeces <-
                #         (1+lagsModelAll[componentsNumberETS+i+1] -
                #              lagsModelAll[componentsNumberETS+i]):lagsModelAll[componentsNumberETS+i+1];
                #     matVt[componentsNumberETS+i,
                #           1:lagsModelAll[componentsNumberETS+i]] <-
                #         (matVt[componentsNumberETS+componentsNumberARIMA, indeces] -
                #              # We need a sum of states here...
                #              matVt[componentsNumberETS+i+1, 1:lagsModelAll[componentsNumberETS+i]]);
                # }

                matVt[componentsNumberETS+nonZeroARI[,2], 1:initialArimaNumber] <-
                    switch(Etype,
                           "A"=arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(B[j+1:initialArimaNumber]),
                           "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(log(B[j+1:initialArimaNumber]))));

                    # switch(Etype,
                    #        "A"=arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(B[j+1:initialArimaNumber]) /
                    #            tail(arimaPolynomials$ariPolynomial,1),
                    #        "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*% t(log(B[j+1:initialArimaNumber])) /
                    #                    tail(arimaPolynomials$ariPolynomial,1)));
                j[] <- j+initialArimaNumber;
            }
            # This is needed in order to propagate initials of ARIMA to all components
            else if(any(c(arEstimate,maEstimate))){
                # if(nrow(nonZeroARI)>0 && nrow(nonZeroARI)>=nrow(nonZeroMA)){
                # if(nrow(nonZeroARI)>0){
                matVt[componentsNumberETS+nonZeroARI[,2], 1:initialArimaNumber] <-
                    switch(Etype,
                           "A"= arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                               t(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]),
                           "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                                       t(log(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]))));

                    # switch(Etype,
                    #        "A"= arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #            t(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]) /
                    #            tail(arimaPolynomials$ariPolynomial,1),
                    #        "M"=exp(arimaPolynomials$ariPolynomial[nonZeroARI[,1]] %*%
                    #                    t(log(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber])) /
                    #                    tail(arimaPolynomials$ariPolynomial,1)));

                # }
                # else{
                #     matVt[componentsNumberETS+nonZeroMA[,2],
                #           1:initialArimaNumber] <-
                #         switch(Etype,
                #                "A"=arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                #                    t(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber]) /
                #                    tail(arimaPolynomials$maPolynomial,1),
                #                "M"=exp(arimaPolynomials$maPolynomial[nonZeroMA[,1]] %*%
                #                            t(log(matVt[componentsNumberETS+componentsNumberARIMA, 1:initialArimaNumber])) /
                #                            tail(arimaPolynomials$maPolynomial,1)));
                # }
            }
        }

        # Initials of the xreg
        if(xregModel && (initialType!="complete") && initialEstimate && initialXregEstimate){
            xregNumberToEstimate <- sum(xregParametersEstimated);
            matVt[componentsNumberETS+componentsNumberARIMA+which(xregParametersEstimated==1),
                  1:lagsModelMax] <- B[j+1:xregNumberToEstimate];
            j[] <- j+xregNumberToEstimate;
            # Normalise initials
            for(i in which(xregParametersMissing!=0)){
                matVt[componentsNumberETS+componentsNumberARIMA+i,
                      1:lagsModelMax] <- -sum(matVt[componentsNumberETS+componentsNumberARIMA+
                                                        which(xregParametersIncluded==xregParametersMissing[i]),
                                                    1:lagsModelMax]);
            }
        }

        # Constant
        if(constantEstimate){
            matVt[componentsNumberETS+componentsNumberARIMA+xregNumber+1,] <- B[j+1];
        }

        return(list(matVt=matVt, matWt=matWt, matF=matF, vecG=vecG, arimaPolynomials=arimaPolynomials));
    }

adamElements <- filler(B,
                               etsModel, Etype, Ttype, Stype, modelIsTrendy, modelIsSeasonal,
                               componentsNumberETS, componentsNumberETSNonSeasonal,
                               componentsNumberETSSeasonal, componentsNumberARIMA,
                               lags, lagsModel, lagsModelMax,
                               matVt, matWt, matF, vecG,
                               persistenceEstimate, persistenceLevelEstimate, persistenceTrendEstimate,
                               persistenceSeasonalEstimate, persistenceXregEstimate,
                               phiEstimate,
                               initialType, initialEstimate,
                               initialLevelEstimate, initialTrendEstimate, initialSeasonalEstimate,
                               initialArimaEstimate, initialXregEstimate,
                               arimaModel, arEstimate, maEstimate, arOrders, iOrders, maOrders,
                               arRequired, maRequired, armaParameters,
                               nonZeroARI, nonZeroMA, arimaPolynomials,
                               xregModel, xregNumber,
                               xregParametersMissing, xregParametersIncluded,
                               xregParametersEstimated, xregParametersPersistence, constantEstimate)
adamElements

$matVt
       [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
level 176.4   NA   NA   NA   NA   NA   NA   NA   NA    NA    NA    NA    NA
      [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
level    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA

$matWt
      level
 [1,]     1
 [2,]     1
 [3,]     1
 [4,]     1
 [5,]     1
 [6,]     1
 [7,]     1
 [8,]     1
 [9,]     1
[10,]     1
[11,]     1
[12,]     1
[13,]     1
[14,]     1
[15,]     1
[16,]     1
[17,]     1
[18,]     1
[19,]     1
[20,]     1
[21,]     1
[22,]     1
[23,]     1
[24,]     1

$matF
     [,1]
[1,]    1

$vecG
      [,1]
alpha  0.1

$arimaPolynomials
NULL



In [ ]:
# this is where it starts

In [7]:
model = "ANN"
lags = [12]

multisteps = False,
lb = None,
ub = None,
maxtime = None,
print_level = 1, # 1 or 0
maxeval = None,
h = 12


# Assume that the model is not provided
# these will be default arguments
profiles_recent_provided = False
profiles_recent_table = None


In [11]:
import time
import inspect

# Start measuring the time of calculations
start_time = time.time()

# Get the call info
cl = inspect.currentframe()

# Record the parental environment. Needed for ARIMA initialization
env = cl.f_back

# Get any additional arguments
ellipsis = {}


# here lines 412 to 524
# what happenes when we provide the parameters of a model
# I assume for now that string is provided for the mdeol

# do the parameter checking
(general, 
 observations_dict,
 persistence_results, 
 initials_results, 
 arima_results, 
 constant_dict, 
 model_type_dict, 
 components_dict, 
 lags_dict, 
 occurrence_dict, 
 phi_dict,
 explanatory_dict,
 params_info) = parameters_checker(ts_df, model=model, lags=lags, h=h)

# I also assume no auto.adam was provided for now and I take the parameters
# what happenes if its auto.adam?

# then lines 4033 to 4070 deals with the occurence model
# this will also wait for a bit 

# then I also skip the regression data on lines 4036


Initial value is not selected. Switching to optimal.


In [12]:
# Lines 4099 to 4138 -> in case we have a model do = estimate
if model_type_dict["model_do"] == "estimate":

    # If this is LASSO/RIDGE with lambda=1, use MSE to estimate initials only
    lambda_original = general['lambda']
    if general['loss'] in ["LASSO", "RIDGE"] and general['lambda'] == 1:
        if model_type_dict["ets_model"]:
            # Pre-set ETS parameters
            persistence_results["persistence_estimate"] = False
            persistence_results["persistence_level_estimate"] = False
            persistence_results["persistence_trend_estimate"] = False
            persistence_results["persistence_seasonal_estimate"] = [False]
            persistence_results["persistence_level"] = 0
            persistence_results["persistence_trend"] = 0 
            persistence_results["persistence_seasonal"] = [0]
            # Phi
            phi_dict["phi_estimate"] = False
            phi_dict["phi"] = 1

        if model_type_dict["xreg_model"]:
            # ETSX parameters
            persistence_results["persistence_xreg_estimate"] = False
            persistence_results["persistence_xreg"] = 0

        if model_type_dict["arima_model"]:
            # Pre-set ARMA parameters
            arima_results["ar_estimate"] = [False]
            arima_results["ma_estimate"] = [False]
            arma_parameters = []
            j = 0
            for i, lag in enumerate(lags_dict["lags"]):
                if arima_results["ar_orders"][i] > 0:
                    arma_parameters.extend([1] * arima_results["ar_orders"][i])
                    j += arima_results["ar_orders"][i]
                if arima_results["ma_orders"][i] > 0:
                    arma_parameters.extend([0] * arima_results["ma_orders"][i])
                    j += arima_results["ma_orders"][i]
            arima_results["arma_parameters"] = arma_parameters

        general['lambda'] = 0


    # Why we estimate first and then have architector and creator?
    adam_estimated = estimator(
        general_dict= general,
        model_type_dict = model_type_dict,
        lags_dict = lags_dict,
        observations_dict = observations_dict,
        arima_dict=arima_results,
        constant_dict=constant_dict,
        explanatory_dict=explanatory_dict,
        profiles_recent_table= profiles_recent_table,
        profiles_recent_provided= profiles_recent_provided,
        persistence_dict=persistence_results,
        initials_dict=initials_results,
        occurrence_dict=occurrence_dict,
        phi_dict=phi_dict,
        components_dict=components_dict,
    )
    # A fix for the special case of lambda==1
    lambda_ = general['lambda']

    # Build the architector
    model_type_dict, components_dict, lags_dict, observations_dict, profile_dict = architector(
    model_type_dict = model_type_dict,
    lags_dict = lags_dict,
    observations_dict = observations_dict,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    explanatory_checked = explanatory_dict,
    profiles_recent_table = profiles_recent_table,
    profiles_recent_provided = profiles_recent_provided
)
    
    # Build the creator
    # Create the matrices for the specific ETS model
    adam_created = creator(
        model_type_dict = model_type_dict,
        lags_dict = lags_dict,
        profiles_dict = profile_dict,
        observations_dict = observations_dict,

        persistence_checked = persistence_results,
        initials_checked = initials_results,
        arima_checked = arima_results,
        constants_checked = constant_dict,
        phi_dict = phi_dict,
        components_dict = components_dict,
        explanatory_checked = explanatory_dict
    )


    # An argument for ic
    ic_selection = ic_function(general['ic'], adam_estimated['log_lik_adam_value'])


# Update parameters number 
# Why we need this for now???
# we will see
n_param_estimated = adam_estimated['n_param_estimated']

# Initialize parameters_number in general if not already present
if 'parameters_number' not in general:
    general['parameters_number'] = params_info['parameters_number']

general['parameters_number'][0][0] = n_param_estimated

# Handle xreg model case (not needed currently )
# If likelihood loss, scale was estimated
if general['loss'] == 'likelihood':
    if len(general['parameters_number'][0]) <= 3:
        general['parameters_number'][0].append(1)
    else:
        general['parameters_number'][0][3] = 1

# Calculate row sums
# Add 4th index if it doesn't exist
if len(general['parameters_number'][0]) <= 4:
    general['parameters_number'][0].append(sum(general['parameters_number'][0][0:4]))
    general['parameters_number'][1].append(sum(general['parameters_number'][1][0:4]))
else:
    general['parameters_number'][0][4] = sum(general['parameters_number'][0][0:4])
    general['parameters_number'][1][4] = sum(general['parameters_number'][1][0:4])

Initial parameters: [1.000e-01 1.764e+02]
<class 'core.estimator.OptimizeResult'>


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [10]:
adam_created

{'mat_vt': array([[176.4,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
           nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
           nan,   nan,   nan,   nan,   nan,   nan,   nan]]),
 'mat_wt': array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 'mat_f': array([[1.]]),
 'vec_g': array([[0.]]),
 'arima_polynomials': None}

In [105]:
# lines 4216 if we have "select"
#elif model_type_dict["model_do"] == "select":
if model_type_dict["model_do"] == "select":
    adam_selected = selector( # function ot yet fully tested
        model_type_dict,
        phi_dict,
        general, 
        lags_dict, 
        observations_dict, 
        arima_results,
        constant_dict,
        explanatory_dict,
        occurrence_dict,
        components_dict,
        profiles_recent_table,
        profiles_recent_provided,
        persistence_results,
        initials_results,

        criterion = "AICc",
        silent = False
    )

    ic_selection = adam_selected['ic_selection']
    results = adam_selected['results']

    best_model = min(ic_selection.items(), key=lambda x: x[1])[0]
    best_id = next(i for i, result in enumerate(results) if result['model'] == best_model)

    model_type_dict = results[best_id]['model_type_dict']
    phi_dict = results[best_id]['phi_dict']
    adam_estimated = results[best_id]['adam_estimated']

    # Build the architector
    model_type_dict, components_dict, lags_dict, observations_dict, profile_dict = architector(
        model_type_dict = model_type_dict,
        lags_dict = lags_dict,
        observations_dict = observations_dict,
        arima_checked = arima_results,
        constants_checked = constant_dict,
        explanatory_checked = explanatory_dict,
        profiles_recent_table = profiles_recent_table,
        profiles_recent_provided = profiles_recent_provided
        )

    # Build the creator
    # Create the matrices for the specific ETS model
    adam_created = creator(
        model_type_dict = model_type_dict,
        lags_dict = lags_dict,
        profiles_dict = profile_dict,
        observations_dict = observations_dict,

        persistence_checked = persistence_results,
        initials_checked = initials_results,
        arima_checked = arima_results,
        constants_checked = constant_dict,
        phi_dict = phi_dict,
        components_dict = components_dict,
        explanatory_checked = explanatory_dict
    )


    # Update parameters number 
    # Why we need this for now???
    # we will see
    n_param_estimated = adam_estimated['n_param_estimated']
    general['parameters_number'] = params_info['parameters_number']

    general['parameters_number'][0][0] = n_param_estimated

    # Handle xreg model case (not needed currently )
    # If likelihood loss, scale was estimated
    if general['loss'] == 'likelihood':
        if len(general['parameters_number'][0]) <= 3:
            general['parameters_number'][0].append(1)
        else:
            general['parameters_number'][0][3] = 1

    # Calculate row sums
    # Add 4th index if it doesn't exist
    if len(general['parameters_number'][0]) <= 4:
        general['parameters_number'][0].append(sum(general['parameters_number'][0][0:4]))
        general['parameters_number'][1].append(sum(general['parameters_number'][1][0:4]))
    else:
        general['parameters_number'][0][4] = sum(general['parameters_number'][0][0:4])
        general['parameters_number'][1][4] = sum(general['parameters_number'][1][0:4])

In [106]:
# if model combine
# lines 4290 

# elif general['model_do'] == 'combine':
if model_type_dict['model_do'] == 'combine':
    model_original = model_type_dict
    # If models pool is not provided, create one
    if general['models_pool'] is None:
        # Define the whole pool of errors
        if not model_type_dict['allow_multiplicative']:
            pool_errors = ['A']
            pool_trends = ['N', 'A', 'Ad'] 
            pool_seasonals = ['N', 'A']
        else:
            pool_errors = ['A', 'M']
            pool_trends = ['N', 'A', 'Ad', 'M', 'Md']
            pool_seasonals = ['N', 'A', 'M']

        # Some preparation variables
        # If error_type is not Z, check on additive errors
        if model_type_dict['error_type'] != 'Z':
            if model_type_dict['error_type'] == 'N':
                pool_errors = ['N']
            elif model_type_dict['error_type'] in ['A', 'X']:
                pool_errors = ['A']
            elif model_type_dict['error_type'] in ['M', 'Y']:
                pool_errors = ['M']

        # If trend_type is not Z, create pool with specified type
        if model_type_dict['trend_type'] != 'Z':
            if model_type_dict['trend_type'] == 'N':
                pool_trends = ['N']
            elif model_type_dict['trend_type'] == 'A':
                pool_trends = ['Ad' if model_type_dict['damped'] else 'A']
            elif model_type_dict['trend_type'] == 'M':
                pool_trends = ['Md' if model_type_dict['damped'] else 'M']
            elif model_type_dict['trend_type'] == 'X':
                pool_trends = ['N', 'A', 'Ad']
            elif model_type_dict['trend_type'] == 'Y':
                pool_trends = ['N', 'M', 'Md']

        # If season_type is not Z, create specific pools
        if model_type_dict['season_type'] != 'Z':
            if model_type_dict['season_type'] == 'N':
                pool_seasonals = ['N']
            elif model_type_dict['season_type'] == 'A':
                pool_seasonals = ['A']
            elif model_type_dict['season_type'] == 'X':
                pool_seasonals = ['N', 'A']
            elif model_type_dict['season_type'] == 'M':
                pool_seasonals = ['M']
            elif model_type_dict['season_type'] == 'Y':
                pool_seasonals = ['N', 'M']

        # Create models pool by combining all possibilities
        general['models_pool'] = [e + t + s for e in pool_errors 
                                for t in pool_trends 
                                for s in pool_seasonals]

    # run the selector
    adam_selected = selector( # function ot yet fully tested
            model_type_dict,
            phi_dict,
            general, 
            lags_dict, 
            observations_dict, 
            arima_results,
            constant_dict,
            explanatory_dict,
            occurrence_dict,
            components_dict,
            profiles_recent_table,
            profiles_recent_provided,
            persistence_results,
            initials_results,

            criterion = "AICc",
            silent = False
        )

    ic_selection = adam_selected['ic_selection']
    results = adam_selected['results']

    ic_best = min(ic_selection.values())
    # Calculate ic weights based on ic_best
    ic_weights = {model: np.exp(-0.5 * (ic - ic_best)) for model, ic in ic_selection.items()}
    weights_sum = sum(ic_weights.values())
    ic_weights = {model: weight/weights_sum for model, weight in ic_weights.items()}

    # Set very small weights to 0 as a failsafe
    ic_weights = {model: 0 if weight < 1e-5 else weight for model, weight in ic_weights.items()}
    weights_sum = sum(ic_weights.values()) 
    ic_weights = {model: weight/weights_sum for model, weight in ic_weights.items()}

    # Add weights to adam_selected dictionary
    adam_selected['ic_weights'] = ic_weights

    # Iterate through all results
    for i in range(len(adam_selected['results'])):
        # Get current result
        result = adam_selected['results'][i]

        model_type_dict, components_dict, lags_dict, observations_dict, profile_dict = architector(
                model_type_dict = result['model_type_dict'],
                lags_dict = lags_dict,
                observations_dict = observations_dict,
                arima_checked = arima_results,
                constants_checked = constant_dict,
                explanatory_checked = explanatory_dict,
                profiles_recent_table = profiles_recent_table,
                profiles_recent_provided = profiles_recent_provided
                )
        # add to adam_selected
        adam_selected['results'][i]['model_type_dict'] = model_type_dict
        adam_selected['results'][i]['components_dict'] = components_dict
        adam_selected['results'][i]['lags_dict'] = lags_dict
        adam_selected['results'][i]['observations_dict'] = observations_dict
        adam_selected['results'][i]['profile_dict'] = profile_dict

        #creator
        adam_created = creator(
            model_type_dict = model_type_dict,
            lags_dict = lags_dict,
            profiles_dict = profile_dict,
            observations_dict = observations_dict,

            persistence_checked = persistence_results,
            initials_checked = initials_results,
            arima_checked = arima_results,
            constants_checked = constant_dict,
            phi_dict = phi_dict,
            components_dict = components_dict,
            explanatory_checked = explanatory_dict
        )

        # add to adam_selected
        adam_selected['results'][i]['adam_created'] = adam_created

        # update parameters number
        n_param_estimated = result['adam_estimated']['n_param_estimated']
        general['parameters_number'] = params_info['parameters_number']

        general['parameters_number'][0][0] = n_param_estimated

        # Handle xreg model case (not needed currently )
        # If likelihood loss, scale was estimated
        if general['loss'] == 'likelihood':
            if len(general['parameters_number'][0]) <= 3:
                general['parameters_number'][0].append(1)
            else:
                general['parameters_number'][0][3] = 1

        # Calculate row sums
        # Add 4th index if it doesn't exist
        if len(general['parameters_number'][0]) <= 4:
            general['parameters_number'][0].append(sum(general['parameters_number'][0][0:4]))
            general['parameters_number'][1].append(sum(general['parameters_number'][1][0:4]))
        else:
            general['parameters_number'][0][4] = sum(general['parameters_number'][0][0:4])
            general['parameters_number'][1][4] = sum(general['parameters_number'][1][0:4])

        # add to adam_selected
        adam_selected['results'][i]['parameters_number'] = general['parameters_number']


In [13]:
# line 4445 if model use
# i will skip this for now

# elif model_type_dict['model_do'] == 'use':

# If the distribution is default, change it according to the error term
if general['distribution'] == "default":
    if general['loss'] == "likelihood":
        if model_type_dict['error_type'] == "A":
            general['distribution_new'] = "dnorm"
        elif model_type_dict['error_type'] == "M":
            general['distribution_new'] = "dgamma"
    elif general['loss'] in ["MAEh", "MACE", "MAE"]:
        general['distribution_new'] = "dlaplace"
    elif general['loss'] in ["HAMh", "CHAM", "HAM"]:
        general['distribution_new'] = "ds"
    elif general['loss'] in ["MSEh", "MSCE", "MSE", "GPL"]:
        general['distribution_new'] = "dnorm"
else:
    general['distribution_new'] = general['distribution']



In [14]:
# Transform everything into appropriate classes

# I assume we have either numpy or pandas series
if isinstance(observations_dict['y_in_sample'], np.ndarray):
    y_in_sample = pd.Series(observations_dict['y_in_sample'], index=pd.date_range(start=observations_dict['y_start'], periods=len(observations_dict['y_in_sample']), freq=observations_dict['frequency']))
    if general['holdout']:
        y_holdout = pd.Series(observations_dict['y_holdout'], 
                            index=pd.date_range(start=observations_dict['y_forecast_start'], 
                                              periods=len(observations_dict['y_holdout']), 
                                              freq=observations_dict['frequency']))
else:
    y_in_sample = observations_dict['y_in_sample'].copy()
    
    if general['holdout']:
        y_holdout = pd.Series(observations_dict['y_holdout'], 
                             index=observations_dict['y_forecast_index'])

In [15]:
# Getting the preparator 
adam_estimated

{'B': array([1.000e-01, 1.764e+02]),
 'CF_value': 1044.3125265799933,
 'n_param_estimated': 2,
 'log_lik_adam_value': {'value': -1044.3125265799933, 'nobs': 24, 'df': 3},
 'arima_polynomials': None}

In [111]:
observations_dict['frequency']

<MonthEnd>

In [ ]:
model = "ANN"
lags = [12]

multisteps = False,
lb = None,
ub = None,
maxtime = None,
print_level = 1, # 1 or 0
maxeval = None,
h = 12


# Assume that the model is not provided
# these will be default arguments
profiles_recent_provided = False
profiles_recent_table = None


In [24]:
lags_dict

{'lags': [1, 12],
 'lags_model': [[1]],
 'lags_model_seasonal': [12],
 'lags_model_arima': [],
 'lags_model_all': [[1]],
 'max_lag': 12,
 'lags_length': 2,
 'lags_model_max': 1}

In [18]:
out = preparator(model_type_dict,
    
    # Components info
    components_dict,
    
    # Lags info
    lags_dict,
    
    # Matrices from creator
    adam_created,
    
    # Parameter dictionaries
    persistence_results,
    initials_results,
    arima_results,
    explanatory_dict,
    phi_dict,
    constant_dict,
    
    # Other parameters
    observations_dict,
    occurrence_dict,
    general,
    profile_dict,
    
    # The parameter vector
    adam_estimated,
    
    # Optional parameters
    bounds="usual",
    other=None
)

In [19]:
out['forecast']

2025-01-31    14.991034
2025-02-28    14.991034
2025-03-31    14.991034
2025-04-30    14.991034
2025-05-31    14.991034
2025-06-30    14.991034
2025-07-31    14.991034
2025-08-31    14.991034
2025-09-30    14.991034
2025-10-31    14.991034
2025-11-30    14.991034
2025-12-31    14.991034
Freq: M, dtype: float64

In [20]:
out['forecast']

2025-01-31    14.991034
2025-02-28    14.991034
2025-03-31    14.991034
2025-04-30    14.991034
2025-05-31    14.991034
2025-06-30    14.991034
2025-07-31    14.991034
2025-08-31    14.991034
2025-09-30    14.991034
2025-10-31    14.991034
2025-11-30    14.991034
2025-12-31    14.991034
Freq: M, dtype: float64

In [26]:
matrices_dict = filler(
        adam_estimated['B'],
        model_type_dict = model_type_dict,
        components_dict = components_dict,
        lags_dict = lags_dict,
        matrices_dict = adam_created,
        persistence_checked = persistence_results,
        initials_checked = initials_results,
        arima_checked = arima_results,
        explanatory_checked = explanatory_dict,
        phi_dict = phi_dict,
        constants_checked = constant_dict
    )
# Write down phi
if phi_dict["phi_estimate"]:
    phi_dict["phi"] = adam_estimated['B'][next(i for i,v in enumerate(B.keys()) if v=="phi")]

# Write down the initials in the recent profile
profile_dict["profiles_recent_table"][:] = matrices_dict['mat_vt'][:, :lags_dict["lags_model_max"]]
profile_dict["profiles_recent_initial"] = matrices_dict['mat_vt'][:, :lags_dict["lags_model_max"]].copy()
adam_fitted = adam_fitter(
    matrices_dict['mat_vt'], matrices_dict['mat_wt'], matrices_dict['mat_f'], matrices_dict['vec_g'],
    lags_dict['lags_model_all'], profile_dict['index_lookup_table'], profile_dict['profiles_recent_table'],
    model_type_dict["error_type"], model_type_dict["trend_type"], model_type_dict["season_type"], 
    components_dict["components_number_ets"], components_dict["components_number_ets_seasonal"],
    components_dict.get("components_number_arima", 0), explanatory_dict["xreg_number"], 
    constant_dict["constant_required"],
    observations_dict["y_in_sample"], observations_dict["ot"], 
    any(x in initials_results["initial_type"] for x in ["complete", "backcasting"])
)
matrices_dict['mat_vt'][:] = adam_fitted["matVt"]
profile_dict["profiles_recent_table"] = adam_fitted["profile"]

# Make sure that there are no negative values in multiplicative components
# This might appear in case of bounds="a"
if model_type_dict["trend_type"] == "M" and (np.any(np.isnan(matrices_dict['mat_vt'][1,:])) or np.any(matrices_dict['mat_vt'][1,:] <= 0)):
    i = np.where(matrices_dict['mat_vt'][1,:] <= 0)[0]
    matrices_dict['mat_vt'][1,i] = 1e-6
    profile_dict["profiles_recent_table"][1,i] = 1e-6

if model_type_dict["season_type"] == "M" and np.all(~np.isnan(matrices_dict['mat_vt'][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],:])) and \
    np.any(matrices_dict['mat_vt'][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],:] <= 0):
    i = np.where(matrices_dict['mat_vt'][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],:] <= 0)[0]
    matrices_dict['mat_vt'][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],i] = 1e-6
    i = np.where(profile_dict["profiles_recent_table"][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],:] <= 0)[0]
    profile_dict["profiles_recent_table"][components_dict["components_number_ets_non_seasonal"]:components_dict["components_number_ets_non_seasonal"]+components_dict["components_number_ets_seasonal"],i] = 1e-6

# Prepare fitted and error with ts / zoo
if not isinstance(observations_dict["y_in_sample"], pd.Series):
    y_fitted = pd.Series(np.full(observations_dict["obs_in_sample"], np.nan), 
                        index=pd.date_range(start=observations_dict["y_start"], 
                                            periods=observations_dict["obs_in_sample"], 
                                            freq=observations_dict["frequency"]))
    errors = pd.Series(np.full(observations_dict["obs_in_sample"], np.nan), 
                        index=pd.date_range(start=observations_dict["y_start"], 
                                        periods=observations_dict["obs_in_sample"], 
                                        freq=observations_dict["frequency"]))
else:
    y_fitted = pd.Series(np.full(observations_dict["obs_in_sample"], np.nan), index=observations_dict["y_in_sample_index"])
    errors = pd.Series(np.full(observations_dict["obs_in_sample"], np.nan), index=observations_dict["y_in_sample_index"])

errors[:] = adam_fitted["errors"].flatten()
y_fitted[:] = adam_fitted["yFitted"].flatten()

 # Check what was returned in the end
if np.any(np.isnan(y_fitted)) or np.any(pd.isna(y_fitted)):
    warnings.warn("Something went wrong in the estimation of the model and NaNs were produced. "
                    "If this is a mixed model, consider using the pure ones instead.")

if occurrence_dict["occurrence_model"]:
    y_fitted[:] = y_fitted * occurrence_dict["p_fitted"]

# Fix the cases, when we have zeroes in the provided occurrence
if occurrence_dict["occurrence"] == "provided":
    y_fitted[~occurrence_dict["ot_logical"]] = y_fitted[~occurrence_dict["ot_logical"]] * occurrence_dict["p_fitted"][~occurrence_dict["ot_logical"]]

# Produce forecasts if the horizon is non-zero
if general["h"] > 0:
    if not isinstance(observations_dict.get("y_in_sample"), pd.Series):
        y_forecast = pd.Series(np.full(general["h"], np.nan), 
                                index=pd.date_range(start=observations_dict["y_forecast_start"], 
                                                periods=general["h"], 
                                                freq=observations_dict["frequency"]))
    else:
        y_forecast = pd.Series(np.full(general["h"], np.nan), 
                                index=observations_dict["y_forecast_index"])
y_forecast[:] = adam_forecaster(
            matrixWt=matrices_dict['mat_wt'][-general["h"]:],
            matrixF=matrices_dict['mat_f'],
            lags=lags_dict["lags_model_all"],
            indexLookupTable=profile_dict["index_lookup_table"],
            profilesRecent=profile_dict["profiles_recent_table"],
            E=model_type_dict["error_type"],
            T=model_type_dict["trend_type"],
            S=model_type_dict["season_type"],
            nNonSeasonal=components_dict["components_number_ets"],
            nSeasonal=components_dict["components_number_ets_seasonal"],
            nArima=components_dict.get("components_number_arima", 0),
            nXreg=explanatory_dict["xreg_number"],
            constant=constant_dict["constant_required"],
            horizon=general["h"]
        ).flatten()

In [49]:


y_forecast

2025-01-31    14.991034
2025-02-28    14.991034
2025-03-31    14.991034
2025-04-30    14.991034
2025-05-31    14.991034
2025-06-30    14.991034
2025-07-31    14.991034
2025-08-31    14.991034
2025-09-30    14.991034
2025-10-31    14.991034
2025-11-30    14.991034
2025-12-31    14.991034
Freq: M, dtype: float64

In [22]:
adam_estimated

{'B': array([1.000e-01, 1.764e+02]),
 'CF_value': 1044.3125265799933,
 'n_param_estimated': 2,
 'log_lik_adam_value': {'value': -1044.3125265799933, 'nobs': 24, 'df': 3},
 'arima_polynomials': None}

In [114]:
adam_estimated

{'B': array([1.000e-01, 1.764e+02]),
 'CF_value': 1044.3125265799933,
 'n_param_estimated': 2,
 'log_lik_adam_value': {'value': -1044.3125265799933, 'nobs': 24, 'df': 3},
 'arima_polynomials': None}

In [107]:
model_type_dict, components_dict, lags_dict, observations_dict, profile_dict = architector(
    model_type_dict = model_type_dict,
    lags_dict = lags_dict,
    observations_dict = observations_dict,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    explanatory_checked = explanatory_dict,
    profiles_recent_table = profiles_recent_table,
    profiles_recent_provided = profiles_recent_provided
)

# Create the matrices for the specific ETS model
adam_created = creator(
    model_type_dict = model_type_dict,
    lags_dict = lags_dict,
    profiles_dict = profile_dict,
    observations_dict = observations_dict,

    persistence_checked = persistence_results,
    initials_checked = initials_results,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    phi_dict = phi_dict,
    components_dict = components_dict,
    explanatory_checked = explanatory_dict
)

# Initialize B
b_values = initialiser(
    model_type_dict = model_type_dict,
    components_dict = components_dict,
    lags_dict = lags_dict,
    adam_created = adam_created,
    persistence_checked = persistence_results,
    initials_checked = initials_results,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    explanatory_checked = explanatory_dict,
    observations_dict = observations_dict,
    bounds = general['bounds'],
    phi_dict = phi_dict,
)
# Create the vector of initials for the optimisation
#if B is None:
B = b_values['B']
#if lb is None:
lb = b_values['Bl']
#if ub is None:
ub = b_values['Bu']


# Companion matrices for the polynomials calculation -> stationarity / stability checks
if model_type_dict['arima_model']:
    # AR polynomials
    ar_polynomial_matrix = np.zeros((np.sum(arima_results['ar_orders']) * lags_dict['lags'], np.sum(arima_results['ar_orders']) * lags_dict['lags']))
    if ar_polynomial_matrix.shape[0] > 1:
        ar_polynomial_matrix[1:, :-1] = np.eye(ar_polynomial_matrix.shape[0] - 1)
    # MA polynomials
    ma_polynomial_matrix = np.zeros((np.sum(arima_results['ma_orders']) * lags_dict['lags'], np.sum(arima_results['ma_orders']) * lags_dict['lags']))
    if ma_polynomial_matrix.shape[0] > 1:
        ma_polynomial_matrix[1:, :-1] = np.eye(ma_polynomial_matrix.shape[0] - 1)
else:
    ma_polynomial_matrix = ar_polynomial_matrix = None


if general['distribution'] == "default":
    if general['loss'] == "likelihood":
        general['distribution_new'] = "dnorm" if model_type_dict['error_type'] == "A" else "dgamma"
    elif general['loss'] in ["MAEh", "MACE", "MAE"]:
        general['distribution_new'] = "dlaplace"
    elif general['loss'] in ["HAMh", "CHAM", "HAM"]:
        general['distribution_new'] = "ds"
    else:
        general['distribution_new'] = "dnorm"
else:
    general['distribution_new'] = general['distribution']

# Handle LASSO/RIDGE denominator calculation
if general['loss'] in ["LASSO", "RIDGE"]:
    if explanatory_dict['xreg_number'] > 0:
        # Calculate standard deviation for each column of matWt
        general['denominator'] = np.std(adam_created['mat_wt'], axis=0)
        # Replace infinite values with 1
        general['denominator'][np.isinf(general['denominator'])] = 1
    else:
        general['denominator'] = None
    # Calculate denominator for y values
    general['y_denominator'] = max(np.std(np.diff(observations_dict['y_in_sample'])), 1)
else:
    general['denominator'] = None
    general['y_denominator'] = None

general['multisteps'] = multisteps


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [98]:
CF(B,  model_type_dict,
        components_dict,
        lags_dict,
        adam_created,
        persistence_results,
        initials_results,
        arima_results,
        explanatory_dict,
        phi_dict,
        constant_dict,
        observations_dict,
        profile_dict,
        general,
        
        bounds = None)

UnboundLocalError: local variable 'CFValue' referenced before assignment

In [108]:
# Create nlopt optimizer object
opt = nlopt.opt(nlopt.LD_SLSQP, len(B))  # Use SLSQP algorithm to match R code

# Set bounds
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)
opt.set_xtol_rel(1e-6)  # Relative tolerance on optimization parameters
opt.set_ftol_rel(1e-6)  # Relative tolerance on function value


In [21]:
profile_dict

{'profiles_recent_table': array([[1.]]),
 'profiles_recent_provided': False,
 'index_lookup_table': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])}

In [22]:
# Work on the filler here
adamElements = filler(B,
                        model_type_dict,
                        components_dict,
                        lags_dict,
                        adam_created,
                        persistence_results,
                        initials_results,
                        arima_results,
                        explanatory_dict,
                        phi_dict,
                        constant_dict)

In [109]:
def objective_wrapper(x, grad):
        return CF(x,
                 model_type_dict,
                 components_dict,
                 lags_dict,
                 adam_created,
                 persistence_results,
                 initials_results,
                 arima_results,
                 explanatory_dict,
                 phi_dict,
                 constant_dict,
                 observations_dict,
                 general,
                 profile_dict)

# Set objective function
opt.set_min_objective(objective_wrapper)


x = opt.optimize(B)
res_fun = opt.last_optimum_value()
res = type('OptimizeResult', (), {
    'x': x,
    'fun': res_fun,
    'success': True
})


KeyError: 'profiles_recent_table'

In [27]:
res.fun

1044.3125265799933

In [ ]:
general, 
 observations_dict,
 persistence_results, 
 initials_results, 
 arima_results, 
 constant_dict, 
 model_type_dict, 
 components_dict, 
 lags_dict, 
 occurrence_dict, 
 phi_dict,
 explanatory_dict

{'distribution': 'default',
 'loss': 'likelihood',
 'outliers': 'ignore',
 'h': 0,
 'holdout': False,
 'ic': 'AICc',
 'bounds': 'usual',
 'model_do': 'estimate',
 'fast': False,
 'models_pool': None,
 'lambda': 1,
 'persistence_params': None,
 'arma_params': None,
 'distribution_new': 'dnorm',
 'denominator': None,
 'y_denominator': None,
 'multisteps': False}

In [171]:
model_type_dict['arima_model']

False

In [70]:
(
    model_type_dict,
        components_dict,
        lags_dict,
        observations_dict,
        profile_dict
) = architector(
    model_type_dict = model_type_dict,
    lags_dict = lags_dict,
    observations_dict = observations_dict,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    explanatory_checked = explanatory_dict,
    #profiles_recent_table = profiles_recent_table,
    #profiles_recent_provided = profiles_recent_provided,

    # not used for now
    #xreg_number = params['xreg_number'],
    #xreg_model = params['xreg_model'],
)

In [64]:
initials_results

{'initial': None,
 'initial_type': 'optimal',
 'initial_estimate': True,
 'initial_level': None,
 'initial_level_estimate': True,
 'initial_trend': None,
 'initial_trend_estimate': True,
 'initial_seasonal': None,
 'initial_seasonal_estimate': True,
 'initial_arima': None,
 'initial_arima_estimate': True,
 'initial_arima_number': 0,
 'initial_xreg_estimate': True,
 'initial_xreg_provided': False}

In [125]:
adam_created = creator(
    model_type_dict = model_type_dict,
    lags_dict = lags_dict,
    profiles_dict = profile_dict,
    observations_dict = observations_dict,

    persistence_checked = persistence_results,
    initials_checked = initials_results,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    phi_dict = phi_dict,
    components_dict = components_dict,
    explanatory_checked = explanatory_dict,
)
adam_created.keys()

/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


dict_keys(['mat_vt', 'mat_wt', 'mat_f', 'vec_g', 'arima_polynomials'])

In [153]:
b_values = initialiser(
    model_type_dict = model_type_dict,
    components_dict = components_dict,
    lags_dict = lags_dict,
    adam_created = adam_created,
    persistence_checked = persistence_results,
    initials_checked = initials_results,
    arima_checked = arima_results,
    constants_checked = constant_dict,
    explanatory_checked = explanatory_dict,
    observations_dict = observations_dict,
    bounds = general['bounds'],
    phi_dict = phi_dict,
)
b_values

{'B': array([1.000e-01, 1.764e+02]),
 'Bl': array([  0., -inf]),
 'Bu': array([ 1., inf]),
 'names': ['alpha', 'level']}

In [152]:
# lines 2647 to 2656 -> skipping them for now
#if b_values['B'] is not None:
 #   if 'names' in b_values:
 #       b_values['B'] = b_values['B'][b_values['names'].isin(b_values['B'].names)]
 #       b_values['B'][:] = b_values['B']
 #   else:
 #       b_values['B'][:] = b_values['B']
 #       b_values['names'] = b_values['B'].names

AttributeError: 'list' object has no attribute 'isin'

In [37]:
adam_created = creator(
    ets_model = params['ets_model'],
    e_type = params['error_type'],
    t_type = params['trend_type'],
    s_type = params['season_type'],
    model_is_trendy = adam_architect['model_is_trendy'],
    model_is_seasonal = adam_architect['model_is_seasonal'],
    lags = params['lags'],
    lags_model = adam_architect['lags_model'],
    lags_model_arima = params['lags_model_arima'],
    lags_model_all = adam_architect['lags_model_all'],
    lags_model_max = adam_architect['lags_model_max'],



)

NameError: name 'creator' is not defined

In [23]:
arch_results = 

SyntaxError: invalid syntax (953245190.py, line 1)